# Author: Makayla McKibben
## Course: DSC540 Data Preparation
## Date: 11.02.2024
## Assignment: Project Milestone 4

In [2]:
import numpy as np
import pandas as pd
import matplotlib as plt
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
import requests
from bs4 import BeautifulSoup
import json
import requests
import itertools

## Milestone 2

In [4]:
# Read data into df and view first 8 rows
webmd_reviews = pd.read_csv('Psychiatric_Drug_Reviews_WebMD.csv')
webmd_reviews.head(8)

,Unnamed: 0,drug_name,date,age,gender,time_on_drug,reviewer_type,condition,rating_overall,rating_effectiveness,rating_ease_of_use,rating_satisfaction,text
0,0,Sertraline Oral,5/12/2024,45-54,Female,1 to less than 2 years,Patient,Posttraumatic Stress Syndrome,5.0,5,5,5,It's almost two years now and it has been so e...
1,1,Sertraline Oral,4/21/2024,35-44,Female,less than 1 month,Patient,Depression,1.0,1,1,1,Iv been on this 4 weeks and iv never felt so a...
2,2,Sertraline Oral,4/16/2024,25-34,Female,2 to less than 5 years,Patient,Repeated Episodes of Anxiety,4.3,4,4,5,Been taking sertraline for past 3 years. Had n...
3,3,Sertraline Oral,4/11/2024,45-54,Male,less than 1 month,Patient,Panic Disorder,1.7,1,3,1,"Of course, take this with a pinch of salt beca..."
4,4,Sertraline Oral,4/8/2024,13-18,Female,NaN,Patient,Major Depressive Disorder,3.0,2,4,3,used for a while saw no effects.
5,5,Sertraline Oral,3/29/2024,45-54,Female,less than 1 month,Patient,Posttraumatic Stress Syndrome,2.3,1,5,1,I had a concussion 9 months before I took this...
6,6,Sertraline Oral,3/27/2024,45-54,Female,1 to 6 months,Patient,Bipolar Depression,5.0,5,5,5,I was skeptical at first because I have been o...
7,7,Sertraline Oral,3/26/2024,12-Jul,Male,1 to 6 months,Caregiver,Repeated Episodes of Anxiety,5.0,5,5,5,My 9yo has moderate ADHD and this year the inc...


### Transformation 1

In [6]:
# Define lambda function and make all text lowercase in a new dataframe
lowered_reviews = webmd_reviews.apply(lambda x: x.astype(str).str.lower())
lowered_reviews

,Unnamed: 0,drug_name,date,age,gender,time_on_drug,reviewer_type,condition,rating_overall,rating_effectiveness,rating_ease_of_use,rating_satisfaction,text
0,0,sertraline oral,5/12/2024,45-54,female,1 to less than 2 years,patient,posttraumatic stress syndrome,5.0,5,5,5,it's almost two years now and it has been so e...
1,1,sertraline oral,4/21/2024,35-44,female,less than 1 month,patient,depression,1.0,1,1,1,iv been on this 4 weeks and iv never felt so a...
2,2,sertraline oral,4/16/2024,25-34,female,2 to less than 5 years,patient,repeated episodes of anxiety,4.3,4,4,5,been taking sertraline for past 3 years. had n...
3,3,sertraline oral,4/11/2024,45-54,male,less than 1 month,patient,panic disorder,1.7,1,3,1,"of course, take this with a pinch of salt beca..."
4,4,sertraline oral,4/8/2024,13-18,female,nan,patient,major depressive disorder,3.0,2,4,3,used for a while saw no effects.
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61315,61315,risperdal oral,9/25/2007,25-34,female,less than 1 month,patient,additional medications to treat depression,1.7,1,3,1,"i got so dizzy, sleepy with slurred speech, af..."
61316,61316,risperdal oral,9/25/2007,12-jul,male,1 to less than 2 years,caregiver,bipolar i disorder with most recent episode mixed,2.0,1,4,1,"while the drug is easy to administer, there wa..."
61317,61317,risperdal oral,9/22/2007,19-24,female,1 to 6 months,patient,bipolar i disorder with most recent episode mixed,3.7,3,5,3,not much significant improvement in mood. made...
61318,61318,risperdal oral,9/19/2007,55-64,male,1 to 6 months,patient,additional medications to treat depression,4.3,4,5,4,"i've had chronic, ""refractory"" depression for ..."


### Transformation 2

In [8]:
# Drop the column that is the same as the index
lowered_reviews = lowered_reviews.drop(lowered_reviews.columns[0], axis = 1)
lowered_reviews

,drug_name,date,age,gender,time_on_drug,reviewer_type,condition,rating_overall,rating_effectiveness,rating_ease_of_use,rating_satisfaction,text
0,sertraline oral,5/12/2024,45-54,female,1 to less than 2 years,patient,posttraumatic stress syndrome,5.0,5,5,5,it's almost two years now and it has been so e...
1,sertraline oral,4/21/2024,35-44,female,less than 1 month,patient,depression,1.0,1,1,1,iv been on this 4 weeks and iv never felt so a...
2,sertraline oral,4/16/2024,25-34,female,2 to less than 5 years,patient,repeated episodes of anxiety,4.3,4,4,5,been taking sertraline for past 3 years. had n...
3,sertraline oral,4/11/2024,45-54,male,less than 1 month,patient,panic disorder,1.7,1,3,1,"of course, take this with a pinch of salt beca..."
4,sertraline oral,4/8/2024,13-18,female,nan,patient,major depressive disorder,3.0,2,4,3,used for a while saw no effects.
...,...,...,...,...,...,...,...,...,...,...,...,...
61315,risperdal oral,9/25/2007,25-34,female,less than 1 month,patient,additional medications to treat depression,1.7,1,3,1,"i got so dizzy, sleepy with slurred speech, af..."
61316,risperdal oral,9/25/2007,12-jul,male,1 to less than 2 years,caregiver,bipolar i disorder with most recent episode mixed,2.0,1,4,1,"while the drug is easy to administer, there wa..."
61317,risperdal oral,9/22/2007,19-24,female,1 to 6 months,patient,bipolar i disorder with most recent episode mixed,3.7,3,5,3,not much significant improvement in mood. made...
61318,risperdal oral,9/19/2007,55-64,male,1 to 6 months,patient,additional medications to treat depression,4.3,4,5,4,"i've had chronic, ""refractory"" depression for ..."


### Transformation 3

In [10]:
# Find all the potential conditions we could match in the dataframe
unique_conditions = lowered_reviews['condition'].unique()
unique_conditions

array(['posttraumatic stress syndrome', 'depression',
       'repeated episodes of anxiety', 'panic disorder',
       'major depressive disorder', 'bipolar depression',
       'depressed mood disorder occurring every year at the same time',
       'anxiousness associated with depression', 'other',
       'obsessive compulsive disorder',
       'extreme apprehension or fear of social interaction',
       'premenstrual disorder with a state of unhappiness',
       'premature ejection of semen', 'problem behavior', 'schizophrenia',
       'osteoporosis', 'aggressive behavior', 'binge eating disorder',
       'acute repetitive seizures', 'anxious', 'muscle spasm',
       'neuropathic pain',
       'attention deficit disorder with hyperactivity', 'stop smoking',
       'manic', 'convulsive seizures', 'facial nerve pain',
       'bipolar i disorder with most recent episode mixed', 'nerve pain',
       'bipolar disorder in remission', 'epileptic seizure', 'tonic',
       'prevention of seizur

### Transformation 4

In [12]:
# Separate only the entries having to do with schizophrenia
cond = 'schizo'
schizo = [c for c in unique_conditions if c.startswith(cond)]
schizo

['schizophrenia', 'schizophrenia with mood changes']

In [13]:
# Keep only those who have schizophrenia with no mood disorder
schizophrenia_reviews = lowered_reviews[(lowered_reviews['condition'] == 'schizophrenia')]
schizophrenia_reviews

,drug_name,date,age,gender,time_on_drug,reviewer_type,condition,rating_overall,rating_effectiveness,rating_ease_of_use,rating_satisfaction,text
618,haloperidol intensol oral,10/5/2010,35-44,male,1 to 6 months,patient,schizophrenia,4.7,5,5,4,it makes me feel restless and more nervous
1766,haloperidol lactate oral,8/26/2009,13-18,female,1 to less than 2 years,caregiver,schizophrenia,2.3,1,4,2,my daughter's halucinations are still present ...
1769,trifluoperazine oral,2/7/2014,45-54,female,less than 1 month,caregiver,schizophrenia,1.0,1,1,1,my son have been diagnosed with schizophrenia ...
4034,chlorpromazine injection,6/19/2009,45-54,female,5 to less than 10 years,patient,schizophrenia,1.0,1,1,1,in fact my mother is using it and it caused se...
4416,loxapine succinate oral,5/4/2011,45-54,male,nan,patient,schizophrenia,5.0,5,5,5,i've been on loxapine since 1988 and i owe my ...
...,...,...,...,...,...,...,...,...,...,...,...,...
61291,risperdal oral,11/23/2007,55-64,female,5 to less than 10 years,patient,schizophrenia,3.3,3,4,3,i have been on risperdal for 7 years and i hav...
61295,risperdal oral,11/18/2007,35-44,female,1 to less than 2 years,caregiver,schizophrenia,4.7,5,5,4,my husband has taken risperdal for a year & a ...
61302,risperdal oral,10/31/2007,25-34,male,2 to less than 5 years,patient,schizophrenia,3.3,3,4,3,the reoccurence is disturbing
61307,risperdal oral,10/19/2007,25-34,female,2 to less than 5 years,patient,schizophrenia,4.3,4,5,4,weight gain is an issue for me. also my eyes s...


In [14]:
# Find the length of our subset
print('This subset has {} entries'.format(len(schizophrenia_reviews)))

This subset has 1096 entries


### Transformation 5

In [16]:
# Keep only those who have schizophrenia and a mood disorder
schizoaffective_reviews = lowered_reviews[(lowered_reviews['condition'] == 'schizophrenia with mood changes')]
schizoaffective_reviews

,drug_name,date,age,gender,time_on_drug,reviewer_type,condition,rating_overall,rating_effectiveness,rating_ease_of_use,rating_satisfaction,text
4602,paliperidone oral,9/30/2018,19-24,female,1 to less than 2 years,patient,schizophrenia with mood changes,5.0,5,5,5,this medication has changed my life i love yhi...
4604,paliperidone oral,9/24/2017,35-44,nan,1 to 6 months,patient,schizophrenia with mood changes,5.0,5,5,5,i've been on paliperidone for about a month no...
4605,paliperidone oral,2/23/2017,35-44,female,1 to 6 months,patient,schizophrenia with mood changes,4.3,4,4,5,i have been on all the other antipsychotics an...
4609,paliperidone oral,5/10/2012,35-44,female,1 to 6 months,patient,schizophrenia with mood changes,4.7,5,5,4,this medication has helped me so much. i have ...
4620,invega oral,1/14/2023,19-24,female,less than 1 month,patient,schizophrenia with mood changes,1.7,1,3,1,"it made my paranoia so much worse, my negative..."
4625,invega oral,1/18/2022,35-44,nan,2 to less than 5 years,patient,schizophrenia with mood changes,4.0,5,5,2,this drug completely cured my paranoid delusio...
4644,invega oral,3/14/2014,55-64,female,1 to less than 2 years,caregiver,schizophrenia with mood changes,1.3,2,1,1,moon face blank look babbling
4658,invega oral,7/12/2012,25-34,female,1 to 6 months,caregiver,schizophrenia with mood changes,4.0,4,4,4,my mother just started taking this after being...
4663,invega oral,5/10/2012,nan,nan,nan,nan,schizophrenia with mood changes,2.3,1,5,1,just prescribed for my daughter and i am wanti...
4675,invega oral,3/29/2011,25-34,male,6 months to less than 1 year,patient,schizophrenia with mood changes,5.0,5,5,5,while taking 6mg of this drug i was doing very...


In [17]:
# Find the length of this subset
print('This subset has {} entries'.format(len(schizoaffective_reviews)))

This subset has 30 entries


### Transformation 6

In [19]:
# Turn nan into NaN
schizoaffective_reviews = schizoaffective_reviews.replace('nan', np.nan)
schizoaffective_reviews

,drug_name,date,age,gender,time_on_drug,reviewer_type,condition,rating_overall,rating_effectiveness,rating_ease_of_use,rating_satisfaction,text
4602,paliperidone oral,9/30/2018,19-24,female,1 to less than 2 years,patient,schizophrenia with mood changes,5.0,5,5,5,this medication has changed my life i love yhi...
4604,paliperidone oral,9/24/2017,35-44,NaN,1 to 6 months,patient,schizophrenia with mood changes,5.0,5,5,5,i've been on paliperidone for about a month no...
4605,paliperidone oral,2/23/2017,35-44,female,1 to 6 months,patient,schizophrenia with mood changes,4.3,4,4,5,i have been on all the other antipsychotics an...
4609,paliperidone oral,5/10/2012,35-44,female,1 to 6 months,patient,schizophrenia with mood changes,4.7,5,5,4,this medication has helped me so much. i have ...
4620,invega oral,1/14/2023,19-24,female,less than 1 month,patient,schizophrenia with mood changes,1.7,1,3,1,"it made my paranoia so much worse, my negative..."
4625,invega oral,1/18/2022,35-44,NaN,2 to less than 5 years,patient,schizophrenia with mood changes,4.0,5,5,2,this drug completely cured my paranoid delusio...
4644,invega oral,3/14/2014,55-64,female,1 to less than 2 years,caregiver,schizophrenia with mood changes,1.3,2,1,1,moon face blank look babbling
4658,invega oral,7/12/2012,25-34,female,1 to 6 months,caregiver,schizophrenia with mood changes,4.0,4,4,4,my mother just started taking this after being...
4663,invega oral,5/10/2012,NaN,NaN,NaN,NaN,schizophrenia with mood changes,2.3,1,5,1,just prescribed for my daughter and i am wanti...
4675,invega oral,3/29/2011,25-34,male,6 months to less than 1 year,patient,schizophrenia with mood changes,5.0,5,5,5,while taking 6mg of this drug i was doing very...


In [20]:
# Find all NaN entries in schizoaffective
schizoaffective_reviews.isna().sum()

drug_name               0
date                    0
age                     1
gender                  6
time_on_drug            1
reviewer_type           1
condition               0
rating_overall          0
rating_effectiveness    0
rating_ease_of_use      0
rating_satisfaction     0
text                    0
dtype: int64

### Transformation 7

In [22]:
# Drop row with too many missing fields
schizoaffective_reviews.dropna(subset = ['age', 'gender', 'time_on_drug', 'reviewer_type'], how ='all', inplace = True)
schizoaffective_reviews

,drug_name,date,age,gender,time_on_drug,reviewer_type,condition,rating_overall,rating_effectiveness,rating_ease_of_use,rating_satisfaction,text
4602,paliperidone oral,9/30/2018,19-24,female,1 to less than 2 years,patient,schizophrenia with mood changes,5.0,5,5,5,this medication has changed my life i love yhi...
4604,paliperidone oral,9/24/2017,35-44,NaN,1 to 6 months,patient,schizophrenia with mood changes,5.0,5,5,5,i've been on paliperidone for about a month no...
4605,paliperidone oral,2/23/2017,35-44,female,1 to 6 months,patient,schizophrenia with mood changes,4.3,4,4,5,i have been on all the other antipsychotics an...
4609,paliperidone oral,5/10/2012,35-44,female,1 to 6 months,patient,schizophrenia with mood changes,4.7,5,5,4,this medication has helped me so much. i have ...
4620,invega oral,1/14/2023,19-24,female,less than 1 month,patient,schizophrenia with mood changes,1.7,1,3,1,"it made my paranoia so much worse, my negative..."
4625,invega oral,1/18/2022,35-44,NaN,2 to less than 5 years,patient,schizophrenia with mood changes,4.0,5,5,2,this drug completely cured my paranoid delusio...
4644,invega oral,3/14/2014,55-64,female,1 to less than 2 years,caregiver,schizophrenia with mood changes,1.3,2,1,1,moon face blank look babbling
4658,invega oral,7/12/2012,25-34,female,1 to 6 months,caregiver,schizophrenia with mood changes,4.0,4,4,4,my mother just started taking this after being...
4675,invega oral,3/29/2011,25-34,male,6 months to less than 1 year,patient,schizophrenia with mood changes,5.0,5,5,5,while taking 6mg of this drug i was doing very...
4687,invega oral,7/24/2010,35-44,female,6 months to less than 1 year,patient,schizophrenia with mood changes,5.0,5,5,5,i've been on invega for now 9 months. and its...


In [23]:
len(schizoaffective_reviews)

29

In [24]:
# Find all NaN entries in schizoaffective
schizoaffective_reviews.isna().sum()

drug_name               0
date                    0
age                     0
gender                  5
time_on_drug            0
reviewer_type           0
condition               0
rating_overall          0
rating_effectiveness    0
rating_ease_of_use      0
rating_satisfaction     0
text                    0
dtype: int64

We successfully removed the row that had four missing values. The remaining data are only missing a single value - all in gender.

### Transformation 8

In [27]:
# Define the function to remove punctuation
def remove_punc(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

In [28]:
schizoaffective_reviews['stripped_text'] = schizoaffective_reviews['text'].apply(remove_punc)
schizoaffective_reviews['stripped_text']

4602    this medication has changed my life i love yhi...
4604    ive been on paliperidone for about a month now...
4605    i have been on all the other antipsychotics an...
4609    this medication has helped me so much i have t...
4620    it made my paranoia so much worse my negative ...
4625    this drug completely cured my paranoid delusio...
4644                       moon face blank look babbling 
4658    my mother just started taking this after being...
4675    while taking 6mg of this drug i was doing very...
4687    ive been on invega for now 9 months  and its e...
4692    was tired and had a racing heartbeat for the f...
4694    i have tried a lot of other meds and this one ...
4697                          a dizzy or tingly sensation
5889    i am schizoeffective prescribed 12mg and get 2...
5894    my son has been taking the injection and i am ...
5909    after taking invega shot monthly for 25 years ...
5913    you will hear voices from a stranger like they...
5915    do not

### Transformation 9

In [30]:
# Set up the stop words values
stop_words = stopwords.words('english')

In [31]:
# Define the function that removes the stop words from the text
def remove_stop_words(text):
    tokenized_words = word_tokenize(text)
    no_stops = [word for word in tokenized_words if word not in stop_words]
    return no_stops

In [32]:
# Remove the stop words
schizoaffective_reviews['stripped_text'] = schizoaffective_reviews['stripped_text'].apply(remove_stop_words)
schizoaffective_reviews['stripped_text']

4602    [medication, changed, life, love, yhis, medica...
4604    [ive, paliperidone, month, scared, reviews, we...
4605    [antipsychotics, one, helped, best, hallucinat...
4609    [medication, helped, much, tried, different, m...
4620    [made, paranoia, much, worse, negative, sympto...
4625    [drug, completely, cured, paranoid, delusions,...
4644                  [moon, face, blank, look, babbling]
4658    [mother, started, taking, countless, medicatio...
4675    [taking, 6mg, drug, well, stress, life, happen...
4687    [ive, invega, 9, months, eliminated, paranoia,...
4692    [tired, racing, heartbeat, first, week, strang...
4694    [tried, lot, meds, one, worked, wonders, drag,...
4697                           [dizzy, tingly, sensation]
5889    [schizoeffective, prescribed, 12mg, get, 2, sc...
5894    [son, taking, injection, concerned, weight, ga...
5909    [taking, invega, shot, monthly, 25, years, str...
5913    [hear, voices, stranger, like, want, take, men...
5915    [repea

### Transformation 10

In [34]:
# Set up stem function
porter = PorterStemmer()

In [35]:
# Define stemming function
def stems(list_of_words):
    stemmed = [porter.stem(word) for word in list_of_words]
    return stemmed

In [36]:
schizoaffective_reviews['stripped_text'] = schizoaffective_reviews['stripped_text'].apply(stems)
schizoaffective_reviews['stripped_text']

4602               [medic, chang, life, love, yhi, medic]
4604    [ive, paliperidon, month, scare, review, weigh...
4605    [antipsychot, one, help, best, hallucin, much,...
4609    [medic, help, much, tri, differ, med, like, ri...
4620    [made, paranoia, much, wors, neg, symptom, bec...
4625    [drug, complet, cure, paranoid, delus, psychot...
4644                     [moon, face, blank, look, babbl]
4658    [mother, start, take, countless, medic, year, ...
4675    [take, 6mg, drug, well, stress, life, happen, ...
4687    [ive, invega, 9, month, elimin, paranoia, audi...
4692    [tire, race, heartbeat, first, week, strang, e...
4694    [tri, lot, med, one, work, wonder, drag, like,...
4697                              [dizzi, tingli, sensat]
5889    [schizoeffect, prescrib, 12mg, get, 2, script,...
5894    [son, take, inject, concern, weight, gain, esp...
5909    [take, invega, shot, monthli, 25, year, straig...
5913    [hear, voic, stranger, like, want, take, menta...
5915    [repea

### Transformation 11

In [38]:
# Join reviews into a single string for each row
schizoaffective_reviews['grouped_text'] = schizoaffective_reviews['stripped_text'].apply(lambda lst: " ". join(lst))
schizoaffective_reviews['grouped_text']

4602                      medic chang life love yhi medic
4604    ive paliperidon month scare review weight gain...
4605    antipsychot one help best hallucin much mood t...
4609    medic help much tri differ med like rispiridon...
4620    made paranoia much wors neg symptom becam wors...
4625    drug complet cure paranoid delus psychot behav...
4644                           moon face blank look babbl
4658    mother start take countless medic year didnt w...
4675    take 6mg drug well stress life happen start be...
4687    ive invega 9 month elimin paranoia audiori hal...
4692    tire race heartbeat first week strang experi c...
4694    tri lot med one work wonder drag like one call...
4697                                  dizzi tingli sensat
5889    schizoeffect prescrib 12mg get 2 script 9mg 3m...
5894    son take inject concern weight gain especi abd...
5909    take invega shot monthli 25 year straight gain...
5913    hear voic stranger like want take mental healt...
5915    repeat

### Transformation 12

In [40]:
# Replace nan with not a number
schizophrenia_reviews = schizophrenia_reviews.replace('nan', np.nan)
schizophrenia_reviews

,drug_name,date,age,gender,time_on_drug,reviewer_type,condition,rating_overall,rating_effectiveness,rating_ease_of_use,rating_satisfaction,text
618,haloperidol intensol oral,10/5/2010,35-44,male,1 to 6 months,patient,schizophrenia,4.7,5,5,4,it makes me feel restless and more nervous
1766,haloperidol lactate oral,8/26/2009,13-18,female,1 to less than 2 years,caregiver,schizophrenia,2.3,1,4,2,my daughter's halucinations are still present ...
1769,trifluoperazine oral,2/7/2014,45-54,female,less than 1 month,caregiver,schizophrenia,1.0,1,1,1,my son have been diagnosed with schizophrenia ...
4034,chlorpromazine injection,6/19/2009,45-54,female,5 to less than 10 years,patient,schizophrenia,1.0,1,1,1,in fact my mother is using it and it caused se...
4416,loxapine succinate oral,5/4/2011,45-54,male,NaN,patient,schizophrenia,5.0,5,5,5,i've been on loxapine since 1988 and i owe my ...
...,...,...,...,...,...,...,...,...,...,...,...,...
61291,risperdal oral,11/23/2007,55-64,female,5 to less than 10 years,patient,schizophrenia,3.3,3,4,3,i have been on risperdal for 7 years and i hav...
61295,risperdal oral,11/18/2007,35-44,female,1 to less than 2 years,caregiver,schizophrenia,4.7,5,5,4,my husband has taken risperdal for a year & a ...
61302,risperdal oral,10/31/2007,25-34,male,2 to less than 5 years,patient,schizophrenia,3.3,3,4,3,the reoccurence is disturbing
61307,risperdal oral,10/19/2007,25-34,female,2 to less than 5 years,patient,schizophrenia,4.3,4,5,4,weight gain is an issue for me. also my eyes s...


In [41]:
# Find all NaN entries in schizophrenia
schizophrenia_reviews.isna().sum()

drug_name                 0
date                      0
age                      47
gender                  110
time_on_drug             50
reviewer_type            36
condition                 0
rating_overall            0
rating_effectiveness      0
rating_ease_of_use        0
rating_satisfaction       0
text                     96
dtype: int64

### Tranformation 13

In [43]:
# Drop row with missing review field. Without the review we can't do sentiment analysis or look for symptoms.
schizophrenia_reviews.dropna(subset = ['text'], how ='all', inplace = True)
schizophrenia_reviews

,drug_name,date,age,gender,time_on_drug,reviewer_type,condition,rating_overall,rating_effectiveness,rating_ease_of_use,rating_satisfaction,text
618,haloperidol intensol oral,10/5/2010,35-44,male,1 to 6 months,patient,schizophrenia,4.7,5,5,4,it makes me feel restless and more nervous
1766,haloperidol lactate oral,8/26/2009,13-18,female,1 to less than 2 years,caregiver,schizophrenia,2.3,1,4,2,my daughter's halucinations are still present ...
1769,trifluoperazine oral,2/7/2014,45-54,female,less than 1 month,caregiver,schizophrenia,1.0,1,1,1,my son have been diagnosed with schizophrenia ...
4034,chlorpromazine injection,6/19/2009,45-54,female,5 to less than 10 years,patient,schizophrenia,1.0,1,1,1,in fact my mother is using it and it caused se...
4416,loxapine succinate oral,5/4/2011,45-54,male,NaN,patient,schizophrenia,5.0,5,5,5,i've been on loxapine since 1988 and i owe my ...
...,...,...,...,...,...,...,...,...,...,...,...,...
61291,risperdal oral,11/23/2007,55-64,female,5 to less than 10 years,patient,schizophrenia,3.3,3,4,3,i have been on risperdal for 7 years and i hav...
61295,risperdal oral,11/18/2007,35-44,female,1 to less than 2 years,caregiver,schizophrenia,4.7,5,5,4,my husband has taken risperdal for a year & a ...
61302,risperdal oral,10/31/2007,25-34,male,2 to less than 5 years,patient,schizophrenia,3.3,3,4,3,the reoccurence is disturbing
61307,risperdal oral,10/19/2007,25-34,female,2 to less than 5 years,patient,schizophrenia,4.3,4,5,4,weight gain is an issue for me. also my eyes s...


In [44]:
len(schizophrenia_reviews)

1000

In [45]:
# Find all NaN entries in schizophrenia
schizophrenia_reviews.isna().sum()

drug_name                 0
date                      0
age                      43
gender                  105
time_on_drug             46
reviewer_type            28
condition                 0
rating_overall            0
rating_effectiveness      0
rating_ease_of_use        0
rating_satisfaction       0
text                      0
dtype: int64

### Transformation 14

In [47]:
schizophrenia_reviews['stripped_text'] = schizophrenia_reviews['text'].apply(remove_punc)
schizophrenia_reviews['stripped_text']

618             it makes me feel restless and more nervous
1766     my daughters halucinations are still present a...
1769     my son have been diagnosed with schizophrenia ...
4034     in fact my mother is using it and it caused se...
4416     ive been on loxapine since 1988 and i owe my s...
                               ...                        
61291    i have been on risperdal for 7 years and i hav...
61295    my husband has taken risperdal for a year  a h...
61302                        the reoccurence is disturbing
61307    weight gain is an issue for me also my eyes sp...
61309    i felt that my medication has helped greatly i...
Name: stripped_text, Length: 1000, dtype: object

### Transformation 15

In [49]:
# Remove the stop words
schizophrenia_reviews['stripped_text'] = schizophrenia_reviews['stripped_text'].apply(remove_stop_words)
schizophrenia_reviews['stripped_text']

618                       [makes, feel, restless, nervous]
1766     [daughters, halucinations, still, present, she...
1769     [son, diagnosed, schizophrenia, resperidone, 3...
4034     [fact, mother, using, caused, sever, side, eff...
4416     [ive, loxapine, since, 1988, owe, survival, ho...
                               ...                        
61291    [risperdal, 7, years, constantly, gained, gain...
61295    [husband, taken, risperdal, year, half, amazin...
61302                            [reoccurence, disturbing]
61307    [weight, gain, issue, also, eyes, spasm, roll,...
61309    [felt, medication, helped, greatly, restoring,...
Name: stripped_text, Length: 1000, dtype: object

### Transformation 16

In [51]:
schizophrenia_reviews['stripped_text'] = schizophrenia_reviews['stripped_text'].apply(stems)
schizophrenia_reviews['stripped_text']

618                         [make, feel, restless, nervou]
1766     [daughter, halucin, still, present, she, high,...
1769     [son, diagnos, schizophrenia, resperidon, 3, w...
4034     [fact, mother, use, caus, sever, side, effect,...
4416     [ive, loxapin, sinc, 1988, owe, surviv, howev,...
                               ...                        
61291    [risperd, 7, year, constantli, gain, gain, wei...
61295    [husband, taken, risperd, year, half, amaz, wo...
61302                                   [reoccur, disturb]
61307    [weight, gain, issu, also, eye, spasm, roll, s...
61309    [felt, medic, help, greatli, restor, mental, w...
Name: stripped_text, Length: 1000, dtype: object

### Transformation 17

In [53]:
# Join reviews into a single string for each row
schizophrenia_reviews['grouped_text'] = schizophrenia_reviews['stripped_text'].apply(lambda lst: " ". join(lst))
schizophrenia_reviews['grouped_text']

618                              make feel restless nervou
1766         daughter halucin still present she high dosag
1769     son diagnos schizophrenia resperidon 3 week dr...
4034     fact mother use caus sever side effect vomitig...
4416     ive loxapin sinc 1988 owe surviv howev never s...
                               ...                        
61291    risperd 7 year constantli gain gain weight loo...
61295    husband taken risperd year half amaz work slee...
61302                                      reoccur disturb
61307    weight gain issu also eye spasm roll strain ex...
61309    felt medic help greatli restor mental well bal...
Name: grouped_text, Length: 1000, dtype: object

### Transformation 18

In [55]:
# Drop row with missing age field. We can't match up with the table with the age field missing.
schizophrenia_reviews.dropna(subset = ['age'], how ='all', inplace = True)
schizophrenia_reviews

,drug_name,date,age,gender,time_on_drug,reviewer_type,condition,rating_overall,rating_effectiveness,rating_ease_of_use,rating_satisfaction,text,stripped_text,grouped_text
618,haloperidol intensol oral,10/5/2010,35-44,male,1 to 6 months,patient,schizophrenia,4.7,5,5,4,it makes me feel restless and more nervous,"[make, feel, restless, nervou]",make feel restless nervou
1766,haloperidol lactate oral,8/26/2009,13-18,female,1 to less than 2 years,caregiver,schizophrenia,2.3,1,4,2,my daughter's halucinations are still present ...,"[daughter, halucin, still, present, she, high,...",daughter halucin still present she high dosag
1769,trifluoperazine oral,2/7/2014,45-54,female,less than 1 month,caregiver,schizophrenia,1.0,1,1,1,my son have been diagnosed with schizophrenia ...,"[son, diagnos, schizophrenia, resperidon, 3, w...",son diagnos schizophrenia resperidon 3 week dr...
4034,chlorpromazine injection,6/19/2009,45-54,female,5 to less than 10 years,patient,schizophrenia,1.0,1,1,1,in fact my mother is using it and it caused se...,"[fact, mother, use, caus, sever, side, effect,...",fact mother use caus sever side effect vomitig...
4416,loxapine succinate oral,5/4/2011,45-54,male,NaN,patient,schizophrenia,5.0,5,5,5,i've been on loxapine since 1988 and i owe my ...,"[ive, loxapin, sinc, 1988, owe, surviv, howev,...",ive loxapin sinc 1988 owe surviv howev never s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61291,risperdal oral,11/23/2007,55-64,female,5 to less than 10 years,patient,schizophrenia,3.3,3,4,3,i have been on risperdal for 7 years and i hav...,"[risperd, 7, year, constantli, gain, gain, wei...",risperd 7 year constantli gain gain weight loo...
61295,risperdal oral,11/18/2007,35-44,female,1 to less than 2 years,caregiver,schizophrenia,4.7,5,5,4,my husband has taken risperdal for a year & a ...,"[husband, taken, risperd, year, half, amaz, wo...",husband taken risperd year half amaz work slee...
61302,risperdal oral,10/31/2007,25-34,male,2 to less than 5 years,patient,schizophrenia,3.3,3,4,3,the reoccurence is disturbing,"[reoccur, disturb]",reoccur disturb
61307,risperdal oral,10/19/2007,25-34,female,2 to less than 5 years,patient,schizophrenia,4.3,4,5,4,weight gain is an issue for me. also my eyes s...,"[weight, gain, issu, also, eye, spasm, roll, s...",weight gain issu also eye spasm roll strain ex...


#### Verify we do not need to do any fuzzy matching for drug names

In [57]:
schizophrenia_reviews['drug_name'].unique()

array(['haloperidol intensol oral', 'haloperidol lactate oral',
       'trifluoperazine oral', 'chlorpromazine injection',
       'loxapine succinate oral', 'chlorpromazine oral',
       'fluphenazine decanoate injection', 'abilify discmelt oral',
       'paliperidone oral', 'invega oral', 'seroquel xr oral',
       'risperidone m-tab oral', 'paliperidone palmitate intramuscular',
       'invega sustenna intramuscular', 'asenapine sublingual',
       'saphris sublingual', 'fanapt oral',
       'olanzapine pamoate intramuscular', 'lurasidone oral',
       'latuda oral', 'saphris (black cherry) sublingual',
       'abilify maintena intramuscular', 'olanzapine oral',
       'invega trinza intramuscular', 'rexulti oral', 'zyprexa oral',
       'cariprazine oral', 'vraylar oral', 'aristada intramuscular',
       'caplyta oral', 'zyprexa zydis oral', 'ziprasidone oral',
       'geodon oral', 'quetiapine oral', 'seroquel oral',
       'prolixin decanoate injection', 'fluphenazine oral', 'hald

In [58]:
schizoaffective_reviews['drug_name'].unique()

array(['paliperidone oral', 'invega oral',
       'invega sustenna intramuscular', 'latuda oral'], dtype=object)

#### Verify all entries were spelled correctly in the gender field

In [60]:
schizophrenia_reviews['gender'].unique()

array(['male', 'female', nan], dtype=object)

In [61]:
schizoaffective_reviews['gender'].unique()

array(['female', nan, 'male'], dtype=object)

### Print Cleaned Schizophrenia Dataset

In [63]:
schizophrenia_reviews

,drug_name,date,age,gender,time_on_drug,reviewer_type,condition,rating_overall,rating_effectiveness,rating_ease_of_use,rating_satisfaction,text,stripped_text,grouped_text
618,haloperidol intensol oral,10/5/2010,35-44,male,1 to 6 months,patient,schizophrenia,4.7,5,5,4,it makes me feel restless and more nervous,"[make, feel, restless, nervou]",make feel restless nervou
1766,haloperidol lactate oral,8/26/2009,13-18,female,1 to less than 2 years,caregiver,schizophrenia,2.3,1,4,2,my daughter's halucinations are still present ...,"[daughter, halucin, still, present, she, high,...",daughter halucin still present she high dosag
1769,trifluoperazine oral,2/7/2014,45-54,female,less than 1 month,caregiver,schizophrenia,1.0,1,1,1,my son have been diagnosed with schizophrenia ...,"[son, diagnos, schizophrenia, resperidon, 3, w...",son diagnos schizophrenia resperidon 3 week dr...
4034,chlorpromazine injection,6/19/2009,45-54,female,5 to less than 10 years,patient,schizophrenia,1.0,1,1,1,in fact my mother is using it and it caused se...,"[fact, mother, use, caus, sever, side, effect,...",fact mother use caus sever side effect vomitig...
4416,loxapine succinate oral,5/4/2011,45-54,male,NaN,patient,schizophrenia,5.0,5,5,5,i've been on loxapine since 1988 and i owe my ...,"[ive, loxapin, sinc, 1988, owe, surviv, howev,...",ive loxapin sinc 1988 owe surviv howev never s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61291,risperdal oral,11/23/2007,55-64,female,5 to less than 10 years,patient,schizophrenia,3.3,3,4,3,i have been on risperdal for 7 years and i hav...,"[risperd, 7, year, constantli, gain, gain, wei...",risperd 7 year constantli gain gain weight loo...
61295,risperdal oral,11/18/2007,35-44,female,1 to less than 2 years,caregiver,schizophrenia,4.7,5,5,4,my husband has taken risperdal for a year & a ...,"[husband, taken, risperd, year, half, amaz, wo...",husband taken risperd year half amaz work slee...
61302,risperdal oral,10/31/2007,25-34,male,2 to less than 5 years,patient,schizophrenia,3.3,3,4,3,the reoccurence is disturbing,"[reoccur, disturb]",reoccur disturb
61307,risperdal oral,10/19/2007,25-34,female,2 to less than 5 years,patient,schizophrenia,4.3,4,5,4,weight gain is an issue for me. also my eyes s...,"[weight, gain, issu, also, eye, spasm, roll, s...",weight gain issu also eye spasm roll strain ex...


### Print Cleaned Schizoaffective Dataset

In [65]:
schizoaffective_reviews

,drug_name,date,age,gender,time_on_drug,reviewer_type,condition,rating_overall,rating_effectiveness,rating_ease_of_use,rating_satisfaction,text,stripped_text,grouped_text
4602,paliperidone oral,9/30/2018,19-24,female,1 to less than 2 years,patient,schizophrenia with mood changes,5.0,5,5,5,this medication has changed my life i love yhi...,"[medic, chang, life, love, yhi, medic]",medic chang life love yhi medic
4604,paliperidone oral,9/24/2017,35-44,NaN,1 to 6 months,patient,schizophrenia with mood changes,5.0,5,5,5,i've been on paliperidone for about a month no...,"[ive, paliperidon, month, scare, review, weigh...",ive paliperidon month scare review weight gain...
4605,paliperidone oral,2/23/2017,35-44,female,1 to 6 months,patient,schizophrenia with mood changes,4.3,4,4,5,i have been on all the other antipsychotics an...,"[antipsychot, one, help, best, hallucin, much,...",antipsychot one help best hallucin much mood t...
4609,paliperidone oral,5/10/2012,35-44,female,1 to 6 months,patient,schizophrenia with mood changes,4.7,5,5,4,this medication has helped me so much. i have ...,"[medic, help, much, tri, differ, med, like, ri...",medic help much tri differ med like rispiridon...
4620,invega oral,1/14/2023,19-24,female,less than 1 month,patient,schizophrenia with mood changes,1.7,1,3,1,"it made my paranoia so much worse, my negative...","[made, paranoia, much, wors, neg, symptom, bec...",made paranoia much wors neg symptom becam wors...
4625,invega oral,1/18/2022,35-44,NaN,2 to less than 5 years,patient,schizophrenia with mood changes,4.0,5,5,2,this drug completely cured my paranoid delusio...,"[drug, complet, cure, paranoid, delus, psychot...",drug complet cure paranoid delus psychot behav...
4644,invega oral,3/14/2014,55-64,female,1 to less than 2 years,caregiver,schizophrenia with mood changes,1.3,2,1,1,moon face blank look babbling,"[moon, face, blank, look, babbl]",moon face blank look babbl
4658,invega oral,7/12/2012,25-34,female,1 to 6 months,caregiver,schizophrenia with mood changes,4.0,4,4,4,my mother just started taking this after being...,"[mother, start, take, countless, medic, year, ...",mother start take countless medic year didnt w...
4675,invega oral,3/29/2011,25-34,male,6 months to less than 1 year,patient,schizophrenia with mood changes,5.0,5,5,5,while taking 6mg of this drug i was doing very...,"[take, 6mg, drug, well, stress, life, happen, ...",take 6mg drug well stress life happen start be...
4687,invega oral,7/24/2010,35-44,female,6 months to less than 1 year,patient,schizophrenia with mood changes,5.0,5,5,5,i've been on invega for now 9 months. and its...,"[ive, invega, 9, month, elimin, paranoia, audi...",ive invega 9 month elimin paranoia audiori hal...


Changing the data was primarily done to split the data by condition, remove rows with missing values that would hinder our analysis when combined with the other data sources, and prepare the reviews for sentiment analysis. There are HIPAA regulations for medical data, but we do not need to worry about them as the data is already anonymous. We've lost some entries by removing rows with missing values, which could influence downstream analysis. There is an assumption that the removed rows were a relatively even distribution of the different data types specified by the column headers and that we aren't skewing our results. By stemming the reviews, we prepare them for sentiment analysis, but they can become incoherent to read when stemmed. We also assume that the program can do the sentiment analysis on 1000 entries well enough and that it is not mandatory for me to go in and do my own sentiment analysis. My dataset came from Kaggle, which came directly from WebMD, where the users first entered their information. There are no ethical concerns about how I obtained the dataset, and I did not run into any ethical dilemmas while working with this dataset.

## Milestone 3

Scrape website for tables.

Use these tables to find frequency of symptoms treated, which binding sites are correlated to the best reviews, which set of pharmacokinetics are correlated to the best reviews?

In [70]:
# Send a GET request to the website
url = 'https://www.ncbi.nlm.nih.gov/books/NBK84656/'
response = requests.get(url)
response

<Response [200]>

In [71]:
# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

In [72]:
# Find the table
tables = soup.find_all('table')
table = tables[0]
table2 = tables[1]

In [73]:
# Extract the data
rows = table.find_all('tr')
data = []
for row in rows:
    headers = row.find_all('th')
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append(cols)

# Extract the data
rows = table2.find_all('tr')
data2 = []
for row in rows:
    headers = row.find_all('th')
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data2.append(cols)

In [74]:
# Print the data
print(data)

[[], ['Chlorpromazine', 'Schizophrenia', 'Adults and children (1–12 years)'], ['Bipolar disorder (mania)'], ['Hyperactivity'], ['Severe behavioral problems'], ['Droperidol', 'Agitation', 'Adults and children'], ['Fluphenazine', 'Psychotic disorders', 'Adults'], ['Haloperidol', 'Schizophrenia', 'Adults'], ['Tourette syndrome'], ['Hyperactivity'], ['Severe childhood behavioral problems'], ['Loxapine', 'Schizophrenia', 'Adults and children ≥12 years'], ['Perphenazine', 'Schizophrenia', 'Adults and children ≥12 years'], ['Pimozide', 'Tourette syndrome', 'Adults and children ≥12 years'], ['Prochlorperazine', 'Schizophrenia', 'Adults and children >2 years and >20 pounds'], ['Generalized nonpsychotic anxiety', 'Adults'], ['Thiothixene', 'Schizophrenia', 'Adults and children ≥12 years'], ['Thioridazine', 'Schizophrenia', 'Adults and children'], ['Trifluoperazine', 'Schizophrenia', 'Adults and children ≥6 years'], ['Generalized nonpsychotic anxiety', 'Adults']]


### Transformation 1

In [76]:
# Fix formatting issues from the pulled data
data = [['Chlorpromazine', 'Schizophrenia', 'Adults and children (1-12 years)'],
        ['Chlorpromazine', 'Bipolar disorder (mania)', 'Adults and children (1-12 years)'],
        ['Chlorpromazine', 'Hyperactivity', 'Adults and children (1-12 years)'],
        ['Chlorpromazine', 'Severe behavioral problems', 'Adults and children (1-12 years)'],
        ['Droperidol', 'Agitation', 'Adults and children'], 
        ['Fluphenazine', 'Psychotic disorders', 'Adults'], 
        ['Haloperidol', 'Schizophrenia', 'Adults'],
        ['Haloperidol', 'Tourette syndrome', 'Adults'],
        ['Haloperidol', 'Hyperactivity', 'Adults'],
        ['Haloperidol', 'Severe childhod behaviroal problems', 'Adults'],
        ['Loxapine', 'Schizophrenia', 'Adults and children >= 12 years'], 
        ['Perphenazine', 'Schizophrenia', 'Adults and children >= 12 years'], 
        ['Pimozide', 'Tourette syndrome', 'Adults and children >= 12 years'], 
        ['Prochlorperazine', 'Schizophrenia', 'Adults and children > 2 years and > 20 pounds'],
        ['Prochlorperazine', 'Generalized nonpsychotic anxiety', 'Adults and children > 2 years and > 20 pounds'],
        ['Thiothixene', 'Schizophrenia', 'Adults and children >= 12 years'], 
        ['Thioridazine', 'Schizophrenia', 'Adults and children >= 12 years'], 
        ['Trifluoperazine', 'Schizophrenia', 'Adults and children >= 6 years'],
        ['Trifluoperazine', 'Generalized nonpsychotic anxiety', 'Adults']]

In [77]:
print(data2)

[[], ['Aripiprazole', 'Schizophrenia', 'Adults and adolescents (13–17 years)'], ['Bipolar disorder (manic/mixed) monotherapy or adjunctive to lithium or valproate', 'Adults and children (10–17 years)'], ['Adjunctive treatment of major depressive disorder', 'Adults'], ['Irritability Associated with autistic disorder', 'Children (6–17 years)'], ['Acute treatment of agitation', 'Adults'], ['Asenapine', 'Acute schizophrenia', 'Adults'], ['Bipolar disorder type 1 (manic/mixed)'], ['Clozapine', 'Treatment resistant schizophrenia', 'Adults'], ['Reduce the risk of suicidal behavior in younger patients with schizophrenia.'], ['Iloperidone', 'Acute schizophrenia', 'Adults'], ['Olanzapine', 'Schizophrenia Bipolar disorder (manic/mixed)', 'Adults and adolescents (13–17 years)'], ['Bipolar disorder', 'Adults'], ['Treatment resistant depression'], ['Agitation associated with schizophrenia and bipolar I mania'], ['Paliperidone', 'Schizophrenia Schizoaffective disorder', 'Adults'], ['Quetiapine', 'Sch

### Transformation 2

In [79]:
# Fix formatting issues from pulling the table
data2 = [['Aripiprazole', 'Schizophrenia', 'Adults and adolescents (13-17 years)'], 
         ['Aripiprazole', 'Bipolar disorder (manic/mixed) monotherapy or adjunctive to lithium or valproate', 
          'Adults and children (10-17 years)'],
         ['Aripiprazole', 'Adjunctive treatment of major depressive disorder',  'Adults'], 
         ['Aripiprazole', 'Irritability associated with autistic disorder', 
         'Children (6-17 years)'],
         ['Aripiprazole', 'Acute treatment of agitation', 'Adults'],
          ['Asenapine', 'Acute schizophrenia', 'Adults'],
         ['Asenapine', 'Bipolar disorder type 1 (manic/mixed)', 'Adults'],
 ['Clozapine', 'Treatment resistant schizophrenia', 'Adults'],
         ['Clozapine', 'Reduce the risk of suicidal behavior in younger patients with schizophrenia', 'Adults'],
 ['Iloperidone', 'Acute schizophrenia', 'Adults'],
 ['Olanzapine', 'Schizophrenia Bipolar disorder (manic/mixed)', 'Adults and adolescents (13-17 years)'],
         ['Olanzapine', 'Bipolar disorder', 'Adults'],
         ['Olanzapine', 'Treatment resistant depression', 'Adults'],
         ['Olanzapine', 'Agitation associated with schizophrenia and bipolar 1 mania', 'Adults'],
 ['Paliperidone', 'Schizophrenia Schizoaffective disorder', 'Adults'],
 ['Quetiapine', 'Schizophrenia','Adults and adolescents (13-17 years)'],
         ['Quetiapine', 'Bipolar disorder (acute manic)', 'Adults, children, and adolescents (10-17 years)'],
         ['Quetiapine', 'Bipolar disorder (depression)', 'Adults'],
         ['Quetiapine', 'Bipolar disorder (maintenance)', 'Adults'],
         ['Quetiapine', 'Adjunctive therapy for major depressive disorder', 'Adults'],
         ['Risperidone', 'Schizophrenia', 'Aduts and adolescents (13-17 years)'], 
         ['Risperidone', 'Bipolar disorder (manic/mixed)', 'Adults and adolescents (10-17 years)'],
         ['Risperidone', 'Irritability associated with autism', 'Children (5-16 years)'],
 ['Ziprasidone', 'Schizophrenia', 'Adults'],
         ['Ziprasidone', 'Bipolar disorder (manic/mixed)', 'Adults'],
         ['Ziprasidone', 'Bipolar disorder (maintenance)', 'Adults'],
         ['Ziprasidone', 'Acute agitation in patients with schizophrenia', 'Adults']]

### Transformation 3

In [81]:
# Convert from list(s) to df of Table 1
data = pd.DataFrame(data)
data

,0,1,2
0,Chlorpromazine,Schizophrenia,Adults and children (1-12 years)
1,Chlorpromazine,Bipolar disorder (mania),Adults and children (1-12 years)
2,Chlorpromazine,Hyperactivity,Adults and children (1-12 years)
3,Chlorpromazine,Severe behavioral problems,Adults and children (1-12 years)
4,Droperidol,Agitation,Adults and children
5,Fluphenazine,Psychotic disorders,Adults
6,Haloperidol,Schizophrenia,Adults
7,Haloperidol,Tourette syndrome,Adults
8,Haloperidol,Hyperactivity,Adults
9,Haloperidol,Severe childhod behaviroal problems,Adults


### Transformation 4

In [83]:
# Convert list(s) from Table 2 to df
data2 = pd.DataFrame(data2)
data2

,0,1,2
0,Aripiprazole,Schizophrenia,Adults and adolescents (13-17 years)
1,Aripiprazole,Bipolar disorder (manic/mixed) monotherapy or ...,Adults and children (10-17 years)
2,Aripiprazole,Adjunctive treatment of major depressive disorder,Adults
3,Aripiprazole,Irritability associated with autistic disorder,Children (6-17 years)
4,Aripiprazole,Acute treatment of agitation,Adults
5,Asenapine,Acute schizophrenia,Adults
6,Asenapine,Bipolar disorder type 1 (manic/mixed),Adults
7,Clozapine,Treatment resistant schizophrenia,Adults
8,Clozapine,Reduce the risk of suicidal behavior in younge...,Adults
9,Iloperidone,Acute schizophrenia,Adults


### Transformation 5

In [85]:
# Changing the name of the first column
data.rename(columns = {0 : 'Generic Name'}, inplace = True)
data

,Generic Name,1,2
0,Chlorpromazine,Schizophrenia,Adults and children (1-12 years)
1,Chlorpromazine,Bipolar disorder (mania),Adults and children (1-12 years)
2,Chlorpromazine,Hyperactivity,Adults and children (1-12 years)
3,Chlorpromazine,Severe behavioral problems,Adults and children (1-12 years)
4,Droperidol,Agitation,Adults and children
5,Fluphenazine,Psychotic disorders,Adults
6,Haloperidol,Schizophrenia,Adults
7,Haloperidol,Tourette syndrome,Adults
8,Haloperidol,Hyperactivity,Adults
9,Haloperidol,Severe childhod behaviroal problems,Adults


### Transformation 6

In [87]:
# Changing the name of the second column
data.rename(columns = {1 : 'Indications'}, inplace = True)
data

,Generic Name,Indications,2
0,Chlorpromazine,Schizophrenia,Adults and children (1-12 years)
1,Chlorpromazine,Bipolar disorder (mania),Adults and children (1-12 years)
2,Chlorpromazine,Hyperactivity,Adults and children (1-12 years)
3,Chlorpromazine,Severe behavioral problems,Adults and children (1-12 years)
4,Droperidol,Agitation,Adults and children
5,Fluphenazine,Psychotic disorders,Adults
6,Haloperidol,Schizophrenia,Adults
7,Haloperidol,Tourette syndrome,Adults
8,Haloperidol,Hyperactivity,Adults
9,Haloperidol,Severe childhod behaviroal problems,Adults


### Transformation 7

In [89]:
# Changing the name of the third column
data.rename(columns = {2 : 'Age Group Approved For'}, inplace = True)
data

,Generic Name,Indications,Age Group Approved For
0,Chlorpromazine,Schizophrenia,Adults and children (1-12 years)
1,Chlorpromazine,Bipolar disorder (mania),Adults and children (1-12 years)
2,Chlorpromazine,Hyperactivity,Adults and children (1-12 years)
3,Chlorpromazine,Severe behavioral problems,Adults and children (1-12 years)
4,Droperidol,Agitation,Adults and children
5,Fluphenazine,Psychotic disorders,Adults
6,Haloperidol,Schizophrenia,Adults
7,Haloperidol,Tourette syndrome,Adults
8,Haloperidol,Hyperactivity,Adults
9,Haloperidol,Severe childhod behaviroal problems,Adults


### Transformation 8

In [91]:
# Changing the name of the first column
data2.rename(columns = {0 : 'Generic Name'}, inplace = True)
data2

,Generic Name,1,2
0,Aripiprazole,Schizophrenia,Adults and adolescents (13-17 years)
1,Aripiprazole,Bipolar disorder (manic/mixed) monotherapy or ...,Adults and children (10-17 years)
2,Aripiprazole,Adjunctive treatment of major depressive disorder,Adults
3,Aripiprazole,Irritability associated with autistic disorder,Children (6-17 years)
4,Aripiprazole,Acute treatment of agitation,Adults
5,Asenapine,Acute schizophrenia,Adults
6,Asenapine,Bipolar disorder type 1 (manic/mixed),Adults
7,Clozapine,Treatment resistant schizophrenia,Adults
8,Clozapine,Reduce the risk of suicidal behavior in younge...,Adults
9,Iloperidone,Acute schizophrenia,Adults


### Transformation 9

In [93]:
# Changing the name of the second column
data2.rename(columns = {1 : 'Indications'}, inplace = True)
data2

,Generic Name,Indications,2
0,Aripiprazole,Schizophrenia,Adults and adolescents (13-17 years)
1,Aripiprazole,Bipolar disorder (manic/mixed) monotherapy or ...,Adults and children (10-17 years)
2,Aripiprazole,Adjunctive treatment of major depressive disorder,Adults
3,Aripiprazole,Irritability associated with autistic disorder,Children (6-17 years)
4,Aripiprazole,Acute treatment of agitation,Adults
5,Asenapine,Acute schizophrenia,Adults
6,Asenapine,Bipolar disorder type 1 (manic/mixed),Adults
7,Clozapine,Treatment resistant schizophrenia,Adults
8,Clozapine,Reduce the risk of suicidal behavior in younge...,Adults
9,Iloperidone,Acute schizophrenia,Adults


### Transformation 10

In [95]:
# Changing the name of the third column
data2.rename(columns = {2 : 'Age Group Approved For'}, inplace = True)
data2

,Generic Name,Indications,Age Group Approved For
0,Aripiprazole,Schizophrenia,Adults and adolescents (13-17 years)
1,Aripiprazole,Bipolar disorder (manic/mixed) monotherapy or ...,Adults and children (10-17 years)
2,Aripiprazole,Adjunctive treatment of major depressive disorder,Adults
3,Aripiprazole,Irritability associated with autistic disorder,Children (6-17 years)
4,Aripiprazole,Acute treatment of agitation,Adults
5,Asenapine,Acute schizophrenia,Adults
6,Asenapine,Bipolar disorder type 1 (manic/mixed),Adults
7,Clozapine,Treatment resistant schizophrenia,Adults
8,Clozapine,Reduce the risk of suicidal behavior in younge...,Adults
9,Iloperidone,Acute schizophrenia,Adults


## Cleaned output from National Library of Medicine

In [97]:
nlm = data
nlm

,Generic Name,Indications,Age Group Approved For
0,Chlorpromazine,Schizophrenia,Adults and children (1-12 years)
1,Chlorpromazine,Bipolar disorder (mania),Adults and children (1-12 years)
2,Chlorpromazine,Hyperactivity,Adults and children (1-12 years)
3,Chlorpromazine,Severe behavioral problems,Adults and children (1-12 years)
4,Droperidol,Agitation,Adults and children
5,Fluphenazine,Psychotic disorders,Adults
6,Haloperidol,Schizophrenia,Adults
7,Haloperidol,Tourette syndrome,Adults
8,Haloperidol,Hyperactivity,Adults
9,Haloperidol,Severe childhod behaviroal problems,Adults


In [98]:
nlm2 = data2
nlm2

,Generic Name,Indications,Age Group Approved For
0,Aripiprazole,Schizophrenia,Adults and adolescents (13-17 years)
1,Aripiprazole,Bipolar disorder (manic/mixed) monotherapy or ...,Adults and children (10-17 years)
2,Aripiprazole,Adjunctive treatment of major depressive disorder,Adults
3,Aripiprazole,Irritability associated with autistic disorder,Children (6-17 years)
4,Aripiprazole,Acute treatment of agitation,Adults
5,Asenapine,Acute schizophrenia,Adults
6,Asenapine,Bipolar disorder type 1 (manic/mixed),Adults
7,Clozapine,Treatment resistant schizophrenia,Adults
8,Clozapine,Reduce the risk of suicidal behavior in younge...,Adults
9,Iloperidone,Acute schizophrenia,Adults


### Find more data to look at

In [100]:
# Send a GET request to the website
url = 'https://en.wikipedia.org/wiki/Antipsychotic'
response = requests.get(url)
response

<Response [200]>

In [101]:
# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

In [102]:
# Find the right tables
tables = soup.find_all('table')
table = tables[2]
table2 = tables[7]
table3 = tables[9]

#### Take a look at tables

In [104]:
tables[2]

<table class="wikitable sortable" style="font-size:small; width:100%;">
<tbody><tr>
<th>Generic name</th>
<th>Class</th>
<th>Type</th>
<th>Brand name(s)</th>
<th>Launch</th>
<th>Developer/Originator(s)</th>
<th>Refs
</th></tr>
<tr>
<td><a href="/wiki/Amisulpride" title="Amisulpride">Amisulpride</a></td>
<td>Benzamide</td>
<td>Disputed</td>
<td>Solian</td>
<td>1986</td>
<td>Sanofi-Synthélabo</td>
<td><sup class="reference" id="cite_ref-Sanchez2012_176-0"><a href="#cite_note-Sanchez2012-176"><span class="cite-bracket">[</span>176<span class="cite-bracket">]</span></a></sup><sup class="reference" id="cite_ref-IUPACFischer2006_177-0"><a href="#cite_note-IUPACFischer2006-177"><span class="cite-bracket">[</span>177<span class="cite-bracket">]</span></a></sup><sup class="reference" id="cite_ref-Lidow2000_178-0"><a href="#cite_note-Lidow2000-178"><span class="cite-bracket">[</span>178<span class="cite-bracket">]</span></a></sup>
</td></tr>
<tr>
<td><a href="/wiki/Aripiprazole" title="Aripipraz

In [105]:
tables[7]

<table class="wikitable sortable" style="font-size:small; width:100%;">
<tbody><tr>
<th colspan="19">K<sub>i</sub> [nM] toward cloned human receptors (unless otherwise specified)<sup class="reference" id="cite_ref-243"><a href="#cite_note-243"><span class="cite-bracket">[</span>Note 1<span class="cite-bracket">]</span></a></sup>
</th></tr>
<tr>
<th>Drug name<sup class="reference" id="cite_ref-GG_129-1"><a href="#cite_note-GG-129"><span class="cite-bracket">[</span>129<span class="cite-bracket">]</span></a></sup><sup class="reference" id="cite_ref-244"><a href="#cite_note-244"><span class="cite-bracket">[</span>243<span class="cite-bracket">]</span></a></sup></th>
<th>SERT</th>
<th>5-HT<sub>1A</sub></th>
<th>5-HT<sub>2A</sub></th>
<th>5-HT<sub>2C</sub></th>
<th>5-HT<sub>6</sub></th>
<th>5-HT<sub>7</sub></th>
<th>α<sub>1A</sub></th>
<th>α<sub>2A</sub></th>
<th>α<sub>2C</sub></th>
<th>NET</th>
<th>D<sub>1</sub></th>
<th>D<sub>2</sub></th>
<th>D<sub>3</sub></th>
<th>D<sub>4</sub></th>
<th>

In [106]:
tables[9]

<table class="wikitable collapsible collapsed" style="font-size:85%; width:100%;">
<tbody><tr>
<th colspan="11" style="font-size:small;">Pharmacokinetics
</th></tr>
<tr>
<td>
<table class="wikitable sortable" style="font-size:small; width:100%;">
<tbody><tr>
<th>Drug<sup class="reference" id="cite_ref-249"><a href="#cite_note-249"><span class="cite-bracket">[</span>248<span class="cite-bracket">]</span></a></sup><sup class="reference" id="cite_ref-250"><a href="#cite_note-250"><span class="cite-bracket">[</span>249<span class="cite-bracket">]</span></a></sup><sup class="reference" id="cite_ref-DM_251-0"><a href="#cite_note-DM-251"><span class="cite-bracket">[</span>250<span class="cite-bracket">]</span></a></sup><sup class="reference" id="cite_ref-252"><a href="#cite_note-252"><span class="cite-bracket">[</span>251<span class="cite-bracket">]</span></a></sup></th>
<th><a href="/wiki/Bioavailability" title="Bioavailability">Bioavailability</a></th>
<th><a href="/wiki/Biological_half-lif

In [107]:
# Extract the data
rows = table.find_all('tr')
data = []
for row in rows:
    headers = row.find_all('th')
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append(cols)

# Extract the data
rows = table2.find_all('tr')
data2 = []
for row in rows:
    headers = row.find_all('th')
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data2.append(cols)

# Extract the data
rows = table3.find_all('tr')
data3 = []
for row in rows:
    headers = row.find_all('th')
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data3.append(cols)

### Transformation 11

In [109]:
# Create df object
data = pd.DataFrame(data)
data

,0,1,2,3,4,5,6
0,None,None,None,None,None,None,None
1,Amisulpride,Benzamide,Disputed,Solian,1986,Sanofi-Synthélabo,[176][177][178]
2,Aripiprazole,Phenylpiperazine/Quinolinone,Atypical,Abilify,2002,Otsuka/Bristol-Myers Squibb,[176][179]
3,Aripiprazole lauroxil,Phenylpiperazine/Quinolinone,Atypical,Aristada,2015,Alkermes,[180]
4,Asenapine,Tricyclic/Dibenzoxapinopyrrole,Atypical,Saphris/Sycrest,2009,Organon/Merck,[176][181]
...,...,...,...,...,...,...,...
59,Triflupromazine,Tricyclic/Phenothiazine,Typical,Vesprin,1957,Bristol-Myers Squibb,[176]
60,Veralipride,Benzamide,Disputed,Agreal,1980,Sanofi-Synthélabo,[176]
61,Ziprasidone,Benzisothiazole,Atypical,Geodon,2000,Pfizer,[176][179]
62,Zotepine,Tricyclic/Dibenzothiepin,Atypical,Zoleptil,1982,Fujisawa,[176][179]


### Transformation 12

In [111]:
# Create df object
data2 = pd.DataFrame(data2)
data2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,Amisulpride,">10,000",">10,000","8,304",">10,000","4,154",73.5,">10,000","1,114","1,540",">10,000",">10,000",2.2,2.4,"2,370",3774.5,">10,000",">10,000",">10,000"
3,Aripiprazole,"1,081",5.6,8.7,22.4,642.4,9.97,25.85,74.1,37.63,2091.5,"1,173.5",1.64,5.35,514,5.3,27.93,"6,778","4,678"
4,Asenapine[244],ND,2.5,0.06,0.03,0.25,0.13,1.2,1.2,1.2,ND,1.4,1.3,0.42,1.1,0.0462,1.0,"8,128","8,128"
5,Blonanserin[245],ND,804,0.812,26.4,41.9,183,26.7 (RB),530 (RC),ND,ND,"1,070",0.142,0.494,150,5.72,765,100,ND
6,Brexpiprazole,ND,0.12,0.47,ND,58,3.7,3.8,15,0.59,ND,160,0.3,1.1,6.3,1.567,19,ND,">10,000"
7,N-DEBN[215],ND,ND,1.28,4.50,5.03,206 (RC),ND,ND,ND,ND,"1,020",1.38,0.23,ND,0.93,ND,ND,ND
8,Chlorpromazine,"1,296","2,115.5",4.5,15.6,17.0,28.4,0.28,184,46,"2,443",76.3,1.40,4.65,5.33,3.21,3.09,32.3,57.0
9,Clozapine,"1,624",123.7,5.35,9.44,13.5,17.95,1.62,37,6.0,"3,168",266.3,157,269.1,26.4,0.0341,1.13,6.17,19.25


### Transformation 13

In [113]:
# Create df object
data3 = pd.DataFrame(data3)
data3

,0,1,2,3,4,5,6,7,8,9,...,343,344,345,346,347,348,349,350,351,352
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Drug[248][249][250][251]\nBioavailability\nt1/...,Amisulpride,48%,12 h,16%,3–4 h,54 ± 4 ng/mL,5.8 L/kg,"Faeces (20%), urine (50%, when given IV)",Oral,...,49%,20 h,98%,2–12 h (mean: 4 h),?,20 L/kg,"Faeces, urine (10%)","Oral, IM (including depot)",CYP2D6,None
2,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Amisulpride,48%,12 h,16%,3–4 h,54 ± 4 ng/mL,5.8 L/kg,"Faeces (20%), urine (50%, when given IV)",Oral,?,...,None,None,None,None,None,None,None,None,None,None
4,Aripiprazole,"87% (Oral), 100% (IM)",75 h (94 h),99%,3–5 h,?,4.9 L/kg,"Faeces (55%), urine (25%)","Oral, IM (including depot)","CYP2D6, CYP3A4",...,None,None,None,None,None,None,None,None,None,None
5,Asenapine,35% (sublingual),24 h,95%,0.5–1.5 h,4 ng/mL,20–25 L/kg,"Urine (50%), faeces (40%)",Sublingual,"CYP1A2, UGT1A4, CYP2D6",...,None,None,None,None,None,None,None,None,None,None
6,Blonanserin[215][216][252],55%,"10.7–16.2 h (single dosing), 67.9 h (repeated ...",≥ 99.7%,1.5–2 h,0.14–0.76 ng/mL (0.57 ng/mL for repeated dosing),8560–9500 L,"Urine (59%), faeces (30%)",Oral,CYP3A4,...,None,None,None,None,None,None,None,None,None,None
7,Chlorpromazine,20%,30 h,92–97%,?,?,20 L/kg,Urine,"Oral, IM, IV",CYP2D6,...,None,None,None,None,None,None,None,None,None,None
8,Clozapine,50–60%,12 h,97%,1.5–2.5 h,102–771 ng/mL,4.67 L/kg,"Urine (50%), faeces (30%)",Oral,"CYP1A2, CYP2D6, CYP3A4",...,None,None,None,None,None,None,None,None,None,None
9,Droperidol,?,2 h (8–12 h),Extensive,60 min (IM),?,"2 L/kg (adults), 0.58 L/kg (children)","Urine (75%), faeces (22%)","IM, IV",?,...,None,None,None,None,None,None,None,None,None,None


### Transformation 14

In [115]:
# Changing the name of the columns
data.rename(columns = {0: 'Generic Name', 1: 'Class', 2 : 'Type', 3: 'Brand Name(s)', 4: 'Launch', 5: 'Developer/Originators'}, inplace = True)
data

,Generic Name,Class,Type,Brand Name(s),Launch,Developer/Originators,6
0,None,None,None,None,None,None,None
1,Amisulpride,Benzamide,Disputed,Solian,1986,Sanofi-Synthélabo,[176][177][178]
2,Aripiprazole,Phenylpiperazine/Quinolinone,Atypical,Abilify,2002,Otsuka/Bristol-Myers Squibb,[176][179]
3,Aripiprazole lauroxil,Phenylpiperazine/Quinolinone,Atypical,Aristada,2015,Alkermes,[180]
4,Asenapine,Tricyclic/Dibenzoxapinopyrrole,Atypical,Saphris/Sycrest,2009,Organon/Merck,[176][181]
...,...,...,...,...,...,...,...
59,Triflupromazine,Tricyclic/Phenothiazine,Typical,Vesprin,1957,Bristol-Myers Squibb,[176]
60,Veralipride,Benzamide,Disputed,Agreal,1980,Sanofi-Synthélabo,[176]
61,Ziprasidone,Benzisothiazole,Atypical,Geodon,2000,Pfizer,[176][179]
62,Zotepine,Tricyclic/Dibenzothiepin,Atypical,Zoleptil,1982,Fujisawa,[176][179]


### Transformation 15

In [117]:
# Drop row we don't need
data.drop(0, inplace = True)
data

,Generic Name,Class,Type,Brand Name(s),Launch,Developer/Originators,6
1,Amisulpride,Benzamide,Disputed,Solian,1986,Sanofi-Synthélabo,[176][177][178]
2,Aripiprazole,Phenylpiperazine/Quinolinone,Atypical,Abilify,2002,Otsuka/Bristol-Myers Squibb,[176][179]
3,Aripiprazole lauroxil,Phenylpiperazine/Quinolinone,Atypical,Aristada,2015,Alkermes,[180]
4,Asenapine,Tricyclic/Dibenzoxapinopyrrole,Atypical,Saphris/Sycrest,2009,Organon/Merck,[176][181]
5,Benperidol,Butyrophenone,Typical,Anquil,1966,Janssen,[176]
...,...,...,...,...,...,...,...
59,Triflupromazine,Tricyclic/Phenothiazine,Typical,Vesprin,1957,Bristol-Myers Squibb,[176]
60,Veralipride,Benzamide,Disputed,Agreal,1980,Sanofi-Synthélabo,[176]
61,Ziprasidone,Benzisothiazole,Atypical,Geodon,2000,Pfizer,[176][179]
62,Zotepine,Tricyclic/Dibenzothiepin,Atypical,Zoleptil,1982,Fujisawa,[176][179]


### Transformation 16

In [119]:
# Drop column we don't need
data.drop(6, inplace = True, axis = 1)
data

,Generic Name,Class,Type,Brand Name(s),Launch,Developer/Originators
1,Amisulpride,Benzamide,Disputed,Solian,1986,Sanofi-Synthélabo
2,Aripiprazole,Phenylpiperazine/Quinolinone,Atypical,Abilify,2002,Otsuka/Bristol-Myers Squibb
3,Aripiprazole lauroxil,Phenylpiperazine/Quinolinone,Atypical,Aristada,2015,Alkermes
4,Asenapine,Tricyclic/Dibenzoxapinopyrrole,Atypical,Saphris/Sycrest,2009,Organon/Merck
5,Benperidol,Butyrophenone,Typical,Anquil,1966,Janssen
...,...,...,...,...,...,...
59,Triflupromazine,Tricyclic/Phenothiazine,Typical,Vesprin,1957,Bristol-Myers Squibb
60,Veralipride,Benzamide,Disputed,Agreal,1980,Sanofi-Synthélabo
61,Ziprasidone,Benzisothiazole,Atypical,Geodon,2000,Pfizer
62,Zotepine,Tricyclic/Dibenzothiepin,Atypical,Zoleptil,1982,Fujisawa


### Transformation 17

In [121]:
# Changing the name of the columns
data2.rename(columns = {0: 'Drug Name', 1: 'SERT', 2 : '5-HT1A', 3: '5-HT2A', 4 : '5-HT2C', 5: '5-HT6', 6: 'HT7', 7: 'alpha1A', 8: 'alpha2A', 9: 'alpha2C', 10: 'NET', 11: 'D1', 12: 'D2', 13: 'D3', 14: 'D4', 15: '5-HT2A/D2', 16: 'H1', 17: 'M1', 18: 'M3'}, inplace = True)
data2

,Drug Name,SERT,5-HT1A,5-HT2A,5-HT2C,5-HT6,HT7,alpha1A,alpha2A,alpha2C,NET,D1,D2,D3,D4,5-HT2A/D2,H1,M1,M3
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,Amisulpride,">10,000",">10,000","8,304",">10,000","4,154",73.5,">10,000","1,114","1,540",">10,000",">10,000",2.2,2.4,"2,370",3774.5,">10,000",">10,000",">10,000"
3,Aripiprazole,"1,081",5.6,8.7,22.4,642.4,9.97,25.85,74.1,37.63,2091.5,"1,173.5",1.64,5.35,514,5.3,27.93,"6,778","4,678"
4,Asenapine[244],ND,2.5,0.06,0.03,0.25,0.13,1.2,1.2,1.2,ND,1.4,1.3,0.42,1.1,0.0462,1.0,"8,128","8,128"
5,Blonanserin[245],ND,804,0.812,26.4,41.9,183,26.7 (RB),530 (RC),ND,ND,"1,070",0.142,0.494,150,5.72,765,100,ND
6,Brexpiprazole,ND,0.12,0.47,ND,58,3.7,3.8,15,0.59,ND,160,0.3,1.1,6.3,1.567,19,ND,">10,000"
7,N-DEBN[215],ND,ND,1.28,4.50,5.03,206 (RC),ND,ND,ND,ND,"1,020",1.38,0.23,ND,0.93,ND,ND,ND
8,Chlorpromazine,"1,296","2,115.5",4.5,15.6,17.0,28.4,0.28,184,46,"2,443",76.3,1.40,4.65,5.33,3.21,3.09,32.3,57.0
9,Clozapine,"1,624",123.7,5.35,9.44,13.5,17.95,1.62,37,6.0,"3,168",266.3,157,269.1,26.4,0.0341,1.13,6.17,19.25


### Transformation 18

In [123]:
# Drop rows we don't need
data2.drop(1, inplace = True)
data2.drop(0, inplace = True)
data2

,Drug Name,SERT,5-HT1A,5-HT2A,5-HT2C,5-HT6,HT7,alpha1A,alpha2A,alpha2C,NET,D1,D2,D3,D4,5-HT2A/D2,H1,M1,M3
2,Amisulpride,">10,000",">10,000","8,304",">10,000","4,154",73.5,">10,000","1,114","1,540",">10,000",">10,000",2.2,2.4,"2,370",3774.5,">10,000",">10,000",">10,000"
3,Aripiprazole,"1,081",5.6,8.7,22.4,642.4,9.97,25.85,74.1,37.63,2091.5,"1,173.5",1.64,5.35,514,5.3,27.93,"6,778","4,678"
4,Asenapine[244],ND,2.5,0.06,0.03,0.25,0.13,1.2,1.2,1.2,ND,1.4,1.3,0.42,1.1,0.0462,1.0,"8,128","8,128"
5,Blonanserin[245],ND,804,0.812,26.4,41.9,183,26.7 (RB),530 (RC),ND,ND,"1,070",0.142,0.494,150,5.72,765,100,ND
6,Brexpiprazole,ND,0.12,0.47,ND,58,3.7,3.8,15,0.59,ND,160,0.3,1.1,6.3,1.567,19,ND,">10,000"
7,N-DEBN[215],ND,ND,1.28,4.50,5.03,206 (RC),ND,ND,ND,ND,"1,020",1.38,0.23,ND,0.93,ND,ND,ND
8,Chlorpromazine,"1,296","2,115.5",4.5,15.6,17.0,28.4,0.28,184,46,"2,443",76.3,1.40,4.65,5.33,3.21,3.09,32.3,57.0
9,Clozapine,"1,624",123.7,5.35,9.44,13.5,17.95,1.62,37,6.0,"3,168",266.3,157,269.1,26.4,0.0341,1.13,6.17,19.25
10,Norclozapine,316.6,13.9,10.9,11.9 (RC),11.6,60.1,104.8,137.6,117.7,493.9,14.3,94.5,153,63.94,0.115,3.4,67.6,95.7
11,cis-Flupenthixol,ND,"8,028",87.5 (HFC),102.2 (RC),ND,ND,ND,ND,ND,ND,3.5,0.35,1.75,66.3,250,0.86,ND,ND


### Transformation 19

In [125]:
# Drop rows we don't need
data2.drop(35, inplace = True)
data2.drop(36, inplace = True)
data2

,Drug Name,SERT,5-HT1A,5-HT2A,5-HT2C,5-HT6,HT7,alpha1A,alpha2A,alpha2C,NET,D1,D2,D3,D4,5-HT2A/D2,H1,M1,M3
2,Amisulpride,">10,000",">10,000","8,304",">10,000","4,154",73.5,">10,000","1,114","1,540",">10,000",">10,000",2.2,2.4,"2,370",3774.5,">10,000",">10,000",">10,000"
3,Aripiprazole,"1,081",5.6,8.7,22.4,642.4,9.97,25.85,74.1,37.63,2091.5,"1,173.5",1.64,5.35,514,5.3,27.93,"6,778","4,678"
4,Asenapine[244],ND,2.5,0.06,0.03,0.25,0.13,1.2,1.2,1.2,ND,1.4,1.3,0.42,1.1,0.0462,1.0,"8,128","8,128"
5,Blonanserin[245],ND,804,0.812,26.4,41.9,183,26.7 (RB),530 (RC),ND,ND,"1,070",0.142,0.494,150,5.72,765,100,ND
6,Brexpiprazole,ND,0.12,0.47,ND,58,3.7,3.8,15,0.59,ND,160,0.3,1.1,6.3,1.567,19,ND,">10,000"
7,N-DEBN[215],ND,ND,1.28,4.50,5.03,206 (RC),ND,ND,ND,ND,"1,020",1.38,0.23,ND,0.93,ND,ND,ND
8,Chlorpromazine,"1,296","2,115.5",4.5,15.6,17.0,28.4,0.28,184,46,"2,443",76.3,1.40,4.65,5.33,3.21,3.09,32.3,57.0
9,Clozapine,"1,624",123.7,5.35,9.44,13.5,17.95,1.62,37,6.0,"3,168",266.3,157,269.1,26.4,0.0341,1.13,6.17,19.25
10,Norclozapine,316.6,13.9,10.9,11.9 (RC),11.6,60.1,104.8,137.6,117.7,493.9,14.3,94.5,153,63.94,0.115,3.4,67.6,95.7
11,cis-Flupenthixol,ND,"8,028",87.5 (HFC),102.2 (RC),ND,ND,ND,ND,ND,ND,3.5,0.35,1.75,66.3,250,0.86,ND,ND


### Transformation 20


In [127]:
# Get headers
headers = data3.iloc[1,0]
headers = headers.split(sep = '\n')

In [128]:
# Find specific headers we need
headers = headers[0:11]
headers

['Drug[248][249][250][251]',
 'Bioavailability',
 't1/2 parent drug (active metabolite)',
 'Protein binding',
 'tmax',
 'Cmax',
 'Vd',
 'Excretion',
 'Routes',
 'Metabolism enzymes[129]',
 'Active metabolites']

In [129]:
# Changing the name of the columns
data3.rename(columns = {0: headers[0], 1: headers[1], 2 : headers[2], 3: headers[3], 4 : headers[4], 5: headers[5], 6: headers[6], 7: headers[7], 8: headers[8], 9: headers[9], 10: headers[10]}, inplace = True)
data3

,Drug[248][249][250][251],Bioavailability,t1/2 parent drug (active metabolite),Protein binding,tmax,Cmax,Vd,Excretion,Routes,Metabolism enzymes[129],...,343,344,345,346,347,348,349,350,351,352
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Drug[248][249][250][251]\nBioavailability\nt1/...,Amisulpride,48%,12 h,16%,3–4 h,54 ± 4 ng/mL,5.8 L/kg,"Faeces (20%), urine (50%, when given IV)",Oral,...,49%,20 h,98%,2–12 h (mean: 4 h),?,20 L/kg,"Faeces, urine (10%)","Oral, IM (including depot)",CYP2D6,None
2,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Amisulpride,48%,12 h,16%,3–4 h,54 ± 4 ng/mL,5.8 L/kg,"Faeces (20%), urine (50%, when given IV)",Oral,?,...,None,None,None,None,None,None,None,None,None,None
4,Aripiprazole,"87% (Oral), 100% (IM)",75 h (94 h),99%,3–5 h,?,4.9 L/kg,"Faeces (55%), urine (25%)","Oral, IM (including depot)","CYP2D6, CYP3A4",...,None,None,None,None,None,None,None,None,None,None
5,Asenapine,35% (sublingual),24 h,95%,0.5–1.5 h,4 ng/mL,20–25 L/kg,"Urine (50%), faeces (40%)",Sublingual,"CYP1A2, UGT1A4, CYP2D6",...,None,None,None,None,None,None,None,None,None,None
6,Blonanserin[215][216][252],55%,"10.7–16.2 h (single dosing), 67.9 h (repeated ...",≥ 99.7%,1.5–2 h,0.14–0.76 ng/mL (0.57 ng/mL for repeated dosing),8560–9500 L,"Urine (59%), faeces (30%)",Oral,CYP3A4,...,None,None,None,None,None,None,None,None,None,None
7,Chlorpromazine,20%,30 h,92–97%,?,?,20 L/kg,Urine,"Oral, IM, IV",CYP2D6,...,None,None,None,None,None,None,None,None,None,None
8,Clozapine,50–60%,12 h,97%,1.5–2.5 h,102–771 ng/mL,4.67 L/kg,"Urine (50%), faeces (30%)",Oral,"CYP1A2, CYP2D6, CYP3A4",...,None,None,None,None,None,None,None,None,None,None
9,Droperidol,?,2 h (8–12 h),Extensive,60 min (IM),?,"2 L/kg (adults), 0.58 L/kg (children)","Urine (75%), faeces (22%)","IM, IV",?,...,None,None,None,None,None,None,None,None,None,None


### Transformation 21

In [131]:
# Drop rows we don't need
data3.drop(0, inplace = True)
data3.drop(1, inplace = True)
data3.drop(2, inplace = True)
data3

,Drug[248][249][250][251],Bioavailability,t1/2 parent drug (active metabolite),Protein binding,tmax,Cmax,Vd,Excretion,Routes,Metabolism enzymes[129],...,343,344,345,346,347,348,349,350,351,352
3,Amisulpride,48%,12 h,16%,3–4 h,54 ± 4 ng/mL,5.8 L/kg,"Faeces (20%), urine (50%, when given IV)",Oral,?,...,None,None,None,None,None,None,None,None,None,None
4,Aripiprazole,"87% (Oral), 100% (IM)",75 h (94 h),99%,3–5 h,?,4.9 L/kg,"Faeces (55%), urine (25%)","Oral, IM (including depot)","CYP2D6, CYP3A4",...,None,None,None,None,None,None,None,None,None,None
5,Asenapine,35% (sublingual),24 h,95%,0.5–1.5 h,4 ng/mL,20–25 L/kg,"Urine (50%), faeces (40%)",Sublingual,"CYP1A2, UGT1A4, CYP2D6",...,None,None,None,None,None,None,None,None,None,None
6,Blonanserin[215][216][252],55%,"10.7–16.2 h (single dosing), 67.9 h (repeated ...",≥ 99.7%,1.5–2 h,0.14–0.76 ng/mL (0.57 ng/mL for repeated dosing),8560–9500 L,"Urine (59%), faeces (30%)",Oral,CYP3A4,...,None,None,None,None,None,None,None,None,None,None
7,Chlorpromazine,20%,30 h,92–97%,?,?,20 L/kg,Urine,"Oral, IM, IV",CYP2D6,...,None,None,None,None,None,None,None,None,None,None
8,Clozapine,50–60%,12 h,97%,1.5–2.5 h,102–771 ng/mL,4.67 L/kg,"Urine (50%), faeces (30%)",Oral,"CYP1A2, CYP2D6, CYP3A4",...,None,None,None,None,None,None,None,None,None,None
9,Droperidol,?,2 h (8–12 h),Extensive,60 min (IM),?,"2 L/kg (adults), 0.58 L/kg (children)","Urine (75%), faeces (22%)","IM, IV",?,...,None,None,None,None,None,None,None,None,None,None
10,Flupentixol,40–55% (Oral),35 h,?,7 days (depot),?,12–14 L/kg,Urine,"Oral, IM (including depot)",?,...,None,None,None,None,None,None,None,None,None,None
11,Fluphenazine,2.7% (Oral),"14–16 h, 14 days (depot)",?,"2 h (Oral), 8–10 h (depot)",?,?,"Urine, faeces","Oral, IM (including depot)",?,...,None,None,None,None,None,None,None,None,None,None
12,Haloperidol,60–70% (Oral),"10–20 h (short-acting IM), 3 weeks (depot)",92%,"2–6 h (Oral), 10–20 min (short-acting IM), 6–7...",?,8–18 L/kg,"Urine (30%), faeces (15%)","Oral, IM, IV",CYP3A4,...,None,None,None,None,None,None,None,None,None,None


### Transformation 22

In [133]:
# Reset index and drop old index values
data.reset_index(inplace = True, drop = True)
data

,Generic Name,Class,Type,Brand Name(s),Launch,Developer/Originators
0,Amisulpride,Benzamide,Disputed,Solian,1986,Sanofi-Synthélabo
1,Aripiprazole,Phenylpiperazine/Quinolinone,Atypical,Abilify,2002,Otsuka/Bristol-Myers Squibb
2,Aripiprazole lauroxil,Phenylpiperazine/Quinolinone,Atypical,Aristada,2015,Alkermes
3,Asenapine,Tricyclic/Dibenzoxapinopyrrole,Atypical,Saphris/Sycrest,2009,Organon/Merck
4,Benperidol,Butyrophenone,Typical,Anquil,1966,Janssen
...,...,...,...,...,...,...
58,Triflupromazine,Tricyclic/Phenothiazine,Typical,Vesprin,1957,Bristol-Myers Squibb
59,Veralipride,Benzamide,Disputed,Agreal,1980,Sanofi-Synthélabo
60,Ziprasidone,Benzisothiazole,Atypical,Geodon,2000,Pfizer
61,Zotepine,Tricyclic/Dibenzothiepin,Atypical,Zoleptil,1982,Fujisawa


### Transformation 23

In [135]:
# Reset index and drop old index values
data2.reset_index(inplace = True, drop = True)
data2

,Drug Name,SERT,5-HT1A,5-HT2A,5-HT2C,5-HT6,HT7,alpha1A,alpha2A,alpha2C,NET,D1,D2,D3,D4,5-HT2A/D2,H1,M1,M3
0,Amisulpride,">10,000",">10,000","8,304",">10,000","4,154",73.5,">10,000","1,114","1,540",">10,000",">10,000",2.2,2.4,"2,370",3774.5,">10,000",">10,000",">10,000"
1,Aripiprazole,"1,081",5.6,8.7,22.4,642.4,9.97,25.85,74.1,37.63,2091.5,"1,173.5",1.64,5.35,514,5.3,27.93,"6,778","4,678"
2,Asenapine[244],ND,2.5,0.06,0.03,0.25,0.13,1.2,1.2,1.2,ND,1.4,1.3,0.42,1.1,0.0462,1.0,"8,128","8,128"
3,Blonanserin[245],ND,804,0.812,26.4,41.9,183,26.7 (RB),530 (RC),ND,ND,"1,070",0.142,0.494,150,5.72,765,100,ND
4,Brexpiprazole,ND,0.12,0.47,ND,58,3.7,3.8,15,0.59,ND,160,0.3,1.1,6.3,1.567,19,ND,">10,000"
5,N-DEBN[215],ND,ND,1.28,4.50,5.03,206 (RC),ND,ND,ND,ND,"1,020",1.38,0.23,ND,0.93,ND,ND,ND
6,Chlorpromazine,"1,296","2,115.5",4.5,15.6,17.0,28.4,0.28,184,46,"2,443",76.3,1.40,4.65,5.33,3.21,3.09,32.3,57.0
7,Clozapine,"1,624",123.7,5.35,9.44,13.5,17.95,1.62,37,6.0,"3,168",266.3,157,269.1,26.4,0.0341,1.13,6.17,19.25
8,Norclozapine,316.6,13.9,10.9,11.9 (RC),11.6,60.1,104.8,137.6,117.7,493.9,14.3,94.5,153,63.94,0.115,3.4,67.6,95.7
9,cis-Flupenthixol,ND,"8,028",87.5 (HFC),102.2 (RC),ND,ND,ND,ND,ND,ND,3.5,0.35,1.75,66.3,250,0.86,ND,ND


### Transformation 24

In [137]:
# Reset index and drop old index values
data3.reset_index(inplace = True, drop = True)
data3

,Drug[248][249][250][251],Bioavailability,t1/2 parent drug (active metabolite),Protein binding,tmax,Cmax,Vd,Excretion,Routes,Metabolism enzymes[129],...,343,344,345,346,347,348,349,350,351,352
0,Amisulpride,48%,12 h,16%,3–4 h,54 ± 4 ng/mL,5.8 L/kg,"Faeces (20%), urine (50%, when given IV)",Oral,?,...,None,None,None,None,None,None,None,None,None,None
1,Aripiprazole,"87% (Oral), 100% (IM)",75 h (94 h),99%,3–5 h,?,4.9 L/kg,"Faeces (55%), urine (25%)","Oral, IM (including depot)","CYP2D6, CYP3A4",...,None,None,None,None,None,None,None,None,None,None
2,Asenapine,35% (sublingual),24 h,95%,0.5–1.5 h,4 ng/mL,20–25 L/kg,"Urine (50%), faeces (40%)",Sublingual,"CYP1A2, UGT1A4, CYP2D6",...,None,None,None,None,None,None,None,None,None,None
3,Blonanserin[215][216][252],55%,"10.7–16.2 h (single dosing), 67.9 h (repeated ...",≥ 99.7%,1.5–2 h,0.14–0.76 ng/mL (0.57 ng/mL for repeated dosing),8560–9500 L,"Urine (59%), faeces (30%)",Oral,CYP3A4,...,None,None,None,None,None,None,None,None,None,None
4,Chlorpromazine,20%,30 h,92–97%,?,?,20 L/kg,Urine,"Oral, IM, IV",CYP2D6,...,None,None,None,None,None,None,None,None,None,None
5,Clozapine,50–60%,12 h,97%,1.5–2.5 h,102–771 ng/mL,4.67 L/kg,"Urine (50%), faeces (30%)",Oral,"CYP1A2, CYP2D6, CYP3A4",...,None,None,None,None,None,None,None,None,None,None
6,Droperidol,?,2 h (8–12 h),Extensive,60 min (IM),?,"2 L/kg (adults), 0.58 L/kg (children)","Urine (75%), faeces (22%)","IM, IV",?,...,None,None,None,None,None,None,None,None,None,None
7,Flupentixol,40–55% (Oral),35 h,?,7 days (depot),?,12–14 L/kg,Urine,"Oral, IM (including depot)",?,...,None,None,None,None,None,None,None,None,None,None
8,Fluphenazine,2.7% (Oral),"14–16 h, 14 days (depot)",?,"2 h (Oral), 8–10 h (depot)",?,?,"Urine, faeces","Oral, IM (including depot)",?,...,None,None,None,None,None,None,None,None,None,None
9,Haloperidol,60–70% (Oral),"10–20 h (short-acting IM), 3 weeks (depot)",92%,"2–6 h (Oral), 10–20 min (short-acting IM), 6–7...",?,8–18 L/kg,"Urine (30%), faeces (15%)","Oral, IM, IV",CYP3A4,...,None,None,None,None,None,None,None,None,None,None


### Transformation 25

In [139]:
# Create array of columns to drop
to_drop = np.arange(11, 353, 1)
to_drop

array([ 11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,
        24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,
        37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,
        50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,
        63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,
        76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,
        89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
       102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
       115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127,
       128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140,
       141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
       154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166,
       167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179,
       180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 19

In [140]:
# Drop unneeded columns
data3.drop(to_drop, axis = 1, inplace = True)
data3

,Drug[248][249][250][251],Bioavailability,t1/2 parent drug (active metabolite),Protein binding,tmax,Cmax,Vd,Excretion,Routes,Metabolism enzymes[129],Active metabolites
0,Amisulpride,48%,12 h,16%,3–4 h,54 ± 4 ng/mL,5.8 L/kg,"Faeces (20%), urine (50%, when given IV)",Oral,?,None
1,Aripiprazole,"87% (Oral), 100% (IM)",75 h (94 h),99%,3–5 h,?,4.9 L/kg,"Faeces (55%), urine (25%)","Oral, IM (including depot)","CYP2D6, CYP3A4",Dehydroaripiprazole
2,Asenapine,35% (sublingual),24 h,95%,0.5–1.5 h,4 ng/mL,20–25 L/kg,"Urine (50%), faeces (40%)",Sublingual,"CYP1A2, UGT1A4, CYP2D6",None
3,Blonanserin[215][216][252],55%,"10.7–16.2 h (single dosing), 67.9 h (repeated ...",≥ 99.7%,1.5–2 h,0.14–0.76 ng/mL (0.57 ng/mL for repeated dosing),8560–9500 L,"Urine (59%), faeces (30%)",Oral,CYP3A4,N-desethylblonanserin
4,Chlorpromazine,20%,30 h,92–97%,?,?,20 L/kg,Urine,"Oral, IM, IV",CYP2D6,Several active metabolites
5,Clozapine,50–60%,12 h,97%,1.5–2.5 h,102–771 ng/mL,4.67 L/kg,"Urine (50%), faeces (30%)",Oral,"CYP1A2, CYP2D6, CYP3A4",Norclozapine
6,Droperidol,?,2 h (8–12 h),Extensive,60 min (IM),?,"2 L/kg (adults), 0.58 L/kg (children)","Urine (75%), faeces (22%)","IM, IV",?,None
7,Flupentixol,40–55% (Oral),35 h,?,7 days (depot),?,12–14 L/kg,Urine,"Oral, IM (including depot)",?,None
8,Fluphenazine,2.7% (Oral),"14–16 h, 14 days (depot)",?,"2 h (Oral), 8–10 h (depot)",?,?,"Urine, faeces","Oral, IM (including depot)",?,None
9,Haloperidol,60–70% (Oral),"10–20 h (short-acting IM), 3 weeks (depot)",92%,"2–6 h (Oral), 10–20 min (short-acting IM), 6–7...",?,8–18 L/kg,"Urine (30%), faeces (15%)","Oral, IM, IV",CYP3A4,None


## Print cleaned Wikipedia datasets

In [142]:
# Change name
wiki = data
wiki

,Generic Name,Class,Type,Brand Name(s),Launch,Developer/Originators
0,Amisulpride,Benzamide,Disputed,Solian,1986,Sanofi-Synthélabo
1,Aripiprazole,Phenylpiperazine/Quinolinone,Atypical,Abilify,2002,Otsuka/Bristol-Myers Squibb
2,Aripiprazole lauroxil,Phenylpiperazine/Quinolinone,Atypical,Aristada,2015,Alkermes
3,Asenapine,Tricyclic/Dibenzoxapinopyrrole,Atypical,Saphris/Sycrest,2009,Organon/Merck
4,Benperidol,Butyrophenone,Typical,Anquil,1966,Janssen
...,...,...,...,...,...,...
58,Triflupromazine,Tricyclic/Phenothiazine,Typical,Vesprin,1957,Bristol-Myers Squibb
59,Veralipride,Benzamide,Disputed,Agreal,1980,Sanofi-Synthélabo
60,Ziprasidone,Benzisothiazole,Atypical,Geodon,2000,Pfizer
61,Zotepine,Tricyclic/Dibenzothiepin,Atypical,Zoleptil,1982,Fujisawa


In [143]:
# Change name
wiki2 = data2
wiki2

,Drug Name,SERT,5-HT1A,5-HT2A,5-HT2C,5-HT6,HT7,alpha1A,alpha2A,alpha2C,NET,D1,D2,D3,D4,5-HT2A/D2,H1,M1,M3
0,Amisulpride,">10,000",">10,000","8,304",">10,000","4,154",73.5,">10,000","1,114","1,540",">10,000",">10,000",2.2,2.4,"2,370",3774.5,">10,000",">10,000",">10,000"
1,Aripiprazole,"1,081",5.6,8.7,22.4,642.4,9.97,25.85,74.1,37.63,2091.5,"1,173.5",1.64,5.35,514,5.3,27.93,"6,778","4,678"
2,Asenapine[244],ND,2.5,0.06,0.03,0.25,0.13,1.2,1.2,1.2,ND,1.4,1.3,0.42,1.1,0.0462,1.0,"8,128","8,128"
3,Blonanserin[245],ND,804,0.812,26.4,41.9,183,26.7 (RB),530 (RC),ND,ND,"1,070",0.142,0.494,150,5.72,765,100,ND
4,Brexpiprazole,ND,0.12,0.47,ND,58,3.7,3.8,15,0.59,ND,160,0.3,1.1,6.3,1.567,19,ND,">10,000"
5,N-DEBN[215],ND,ND,1.28,4.50,5.03,206 (RC),ND,ND,ND,ND,"1,020",1.38,0.23,ND,0.93,ND,ND,ND
6,Chlorpromazine,"1,296","2,115.5",4.5,15.6,17.0,28.4,0.28,184,46,"2,443",76.3,1.40,4.65,5.33,3.21,3.09,32.3,57.0
7,Clozapine,"1,624",123.7,5.35,9.44,13.5,17.95,1.62,37,6.0,"3,168",266.3,157,269.1,26.4,0.0341,1.13,6.17,19.25
8,Norclozapine,316.6,13.9,10.9,11.9 (RC),11.6,60.1,104.8,137.6,117.7,493.9,14.3,94.5,153,63.94,0.115,3.4,67.6,95.7
9,cis-Flupenthixol,ND,"8,028",87.5 (HFC),102.2 (RC),ND,ND,ND,ND,ND,ND,3.5,0.35,1.75,66.3,250,0.86,ND,ND


In [144]:
# Change name
wiki3 = data3
wiki3

,Drug[248][249][250][251],Bioavailability,t1/2 parent drug (active metabolite),Protein binding,tmax,Cmax,Vd,Excretion,Routes,Metabolism enzymes[129],Active metabolites
0,Amisulpride,48%,12 h,16%,3–4 h,54 ± 4 ng/mL,5.8 L/kg,"Faeces (20%), urine (50%, when given IV)",Oral,?,None
1,Aripiprazole,"87% (Oral), 100% (IM)",75 h (94 h),99%,3–5 h,?,4.9 L/kg,"Faeces (55%), urine (25%)","Oral, IM (including depot)","CYP2D6, CYP3A4",Dehydroaripiprazole
2,Asenapine,35% (sublingual),24 h,95%,0.5–1.5 h,4 ng/mL,20–25 L/kg,"Urine (50%), faeces (40%)",Sublingual,"CYP1A2, UGT1A4, CYP2D6",None
3,Blonanserin[215][216][252],55%,"10.7–16.2 h (single dosing), 67.9 h (repeated ...",≥ 99.7%,1.5–2 h,0.14–0.76 ng/mL (0.57 ng/mL for repeated dosing),8560–9500 L,"Urine (59%), faeces (30%)",Oral,CYP3A4,N-desethylblonanserin
4,Chlorpromazine,20%,30 h,92–97%,?,?,20 L/kg,Urine,"Oral, IM, IV",CYP2D6,Several active metabolites
5,Clozapine,50–60%,12 h,97%,1.5–2.5 h,102–771 ng/mL,4.67 L/kg,"Urine (50%), faeces (30%)",Oral,"CYP1A2, CYP2D6, CYP3A4",Norclozapine
6,Droperidol,?,2 h (8–12 h),Extensive,60 min (IM),?,"2 L/kg (adults), 0.58 L/kg (children)","Urine (75%), faeces (22%)","IM, IV",?,None
7,Flupentixol,40–55% (Oral),35 h,?,7 days (depot),?,12–14 L/kg,Urine,"Oral, IM (including depot)",?,None
8,Fluphenazine,2.7% (Oral),"14–16 h, 14 days (depot)",?,"2 h (Oral), 8–10 h (depot)",?,?,"Urine, faeces","Oral, IM (including depot)",?,None
9,Haloperidol,60–70% (Oral),"10–20 h (short-acting IM), 3 weeks (depot)",92%,"2–6 h (Oral), 10–20 min (short-acting IM), 6–7...",?,8–18 L/kg,"Urine (30%), faeces (15%)","Oral, IM, IV",CYP3A4,None


## Milestone 4

In [146]:
# Create empty lists
all_names = []
all_urls = []
all_desc = []

In [147]:
# Set the url to use in the get request
api = str('https://sandbox.api.service.nhs.uk/nhs-website-content/medicines/')
# Get request
response = requests.get(api)
# Json formatted data
json_resp = response.json()
# Create list of dictionaries in the significantLink section
list_of_dict = json_resp['significantLink']
list_of_dict
# Iterate through the list of dictionaries
for dict in list_of_dict:
    # Grab the data with the specified header
    name_vals = list(map(lambda x: x['name'], list_of_dict))
    url_vals = list(map(lambda x: x['url'], list_of_dict))
    desc_vals = list(map(lambda x: x['description'], list_of_dict))

# Append values to our lists
all_names.append(name_vals)
all_urls.append(url_vals)
all_desc.append(desc_vals)

# Display values
name_vals, url_vals, desc_vals        

(['Aciclovir (Zovirax)',
  'Acrivastine',
  'Adalimumab',
  'Alendronic acid',
  'Allopurinol',
  'Alogliptin',
  'Amitriptyline for depression',
  'Amitriptyline for pain and migraine',
  'Amlodipine',
  'Amoxicillin',
  'Anastrozole',
  'Apixaban',
  'Aripiprazole',
  'Aspirin for pain relief',
  'Atenolol',
  'Atorvastatin',
  'Azathioprine',
  'Azithromycin',
  'Baclofen',
  'Beclometasone inhalers',
  'Beclometasone nasal spray',
  'Beclometasone skin creams',
  'Beclometasone tablets',
  'Bendroflumethiazide',
  'Benzoyl peroxide'],
 ['https://api.service.nhs.uk/nhs-website-content/medicines/aciclovir/',
  'https://api.service.nhs.uk/nhs-website-content/medicines/acrivastine/',
  'https://api.service.nhs.uk/nhs-website-content/medicines/adalimumab/',
  'https://api.service.nhs.uk/nhs-website-content/medicines/alendronic-acid/',
  'https://api.service.nhs.uk/nhs-website-content/medicines/allopurinol/',
  'https://api.service.nhs.uk/nhs-website-content/medicines/alogliptin/',
  'ht

In [148]:
# Create for loop to traverse all the pages of the API
for i in range(1, 11):
    # Pass the parameters needed to call the API data
    headers = {'Content-Type': 'application/json', 'subscription-key': 'ef9f420ba7764045aefce8f0ed00940a', 'apikey': '6NhWXS3wtsYVt23jFQi1AhyojA4FEDYp'}
    # Make the url we'll pull   
    api = 'https://api.nhs.uk/medicines/?page=' + str(i)
    # Get request with set parameters
    response = requests.get(api, headers = headers)
    # Set the Json data
    json_resp = response.json()
    json_resp
    # Create list of dictionaries in significantLink 
    list_of_dict = json_resp['significantLink']
    # Find the values of the specified parameters using a lambda function
    name_vals = list(map(lambda x: x['name'], list_of_dict))
    url_vals = list(map(lambda x: x['url'], list_of_dict))
    desc_vals = list(map(lambda x: x['description'], list_of_dict))
    # Append to our lists
    all_names.append(name_vals)
    all_urls.append(url_vals)
    all_desc.append(desc_vals)
    
# Display our lists
all_names, all_urls, all_desc  

([['Aciclovir (Zovirax)',
   'Acrivastine',
   'Adalimumab',
   'Alendronic acid',
   'Allopurinol',
   'Alogliptin',
   'Amitriptyline for depression',
   'Amitriptyline for pain and migraine',
   'Amlodipine',
   'Amoxicillin',
   'Anastrozole',
   'Apixaban',
   'Aripiprazole',
   'Aspirin for pain relief',
   'Atenolol',
   'Atorvastatin',
   'Azathioprine',
   'Azithromycin',
   'Baclofen',
   'Beclometasone inhalers',
   'Beclometasone nasal spray',
   'Beclometasone skin creams',
   'Beclometasone tablets',
   'Bendroflumethiazide',
   'Benzoyl peroxide'],
  ['Aciclovir (Zovirax)',
   'Acrivastine',
   'Adalimumab',
   'Alendronic acid',
   'Allopurinol',
   'Alogliptin',
   'Amitriptyline for depression',
   'Amitriptyline for pain and migraine',
   'Amlodipine',
   'Amoxicillin',
   'Anastrozole',
   'Apixaban',
   'Aripiprazole',
   'Aspirin for pain relief',
   'Atenolol',
   'Atorvastatin',
   'Azathioprine',
   'Azithromycin',
   'Baclofen',
   'Beclometasone inhalers',
  

In [149]:
# Create empty list
med_names = []
# Use for loop to un-nest lists
for inner_list in all_names:
    med_names.extend(inner_list)
med_names

['Aciclovir (Zovirax)',
 'Acrivastine',
 'Adalimumab',
 'Alendronic acid',
 'Allopurinol',
 'Alogliptin',
 'Amitriptyline for depression',
 'Amitriptyline for pain and migraine',
 'Amlodipine',
 'Amoxicillin',
 'Anastrozole',
 'Apixaban',
 'Aripiprazole',
 'Aspirin for pain relief',
 'Atenolol',
 'Atorvastatin',
 'Azathioprine',
 'Azithromycin',
 'Baclofen',
 'Beclometasone inhalers',
 'Beclometasone nasal spray',
 'Beclometasone skin creams',
 'Beclometasone tablets',
 'Bendroflumethiazide',
 'Benzoyl peroxide',
 'Aciclovir (Zovirax)',
 'Acrivastine',
 'Adalimumab',
 'Alendronic acid',
 'Allopurinol',
 'Alogliptin',
 'Amitriptyline for depression',
 'Amitriptyline for pain and migraine',
 'Amlodipine',
 'Amoxicillin',
 'Anastrozole',
 'Apixaban',
 'Aripiprazole',
 'Aspirin for pain relief',
 'Atenolol',
 'Atorvastatin',
 'Azathioprine',
 'Azithromycin',
 'Baclofen',
 'Beclometasone inhalers',
 'Beclometasone nasal spray',
 'Beclometasone skin cream',
 'Beclometasone tablets',
 'Bendro

In [150]:
# Create empty list
url_medspec = []
# Use for loop to un-nest the lists
for inner_list in all_urls:
    url_medspec.extend(inner_list)
url_medspec

['https://api.service.nhs.uk/nhs-website-content/medicines/aciclovir/',
 'https://api.service.nhs.uk/nhs-website-content/medicines/acrivastine/',
 'https://api.service.nhs.uk/nhs-website-content/medicines/adalimumab/',
 'https://api.service.nhs.uk/nhs-website-content/medicines/alendronic-acid/',
 'https://api.service.nhs.uk/nhs-website-content/medicines/allopurinol/',
 'https://api.service.nhs.uk/nhs-website-content/medicines/alogliptin/',
 'https://api.service.nhs.uk/nhs-website-content/medicines/amitriptyline-for-depression/',
 'https://api.service.nhs.uk/nhs-website-content/medicines/amitriptyline-for-pain/',
 'https://api.service.nhs.uk/nhs-website-content/medicines/amlodipine/',
 'https://api.service.nhs.uk/nhs-website-content/medicines/amoxicillin/',
 'https://api.service.nhs.uk/nhs-website-content/medicines/anastrozole/',
 'https://api.service.nhs.uk/nhs-website-content/medicines/apixaban/',
 'https://api.service.nhs.uk/nhs-website-content/medicines/aripiprazole/',
 'https://api

In [151]:
# Create empty list
new_url =[]
# Use for loop to make sure all urls are formatted appropriately
for url in url_medspec:
    left, right, = url.split(sep = '//')
    new = (left + '//int.' + right)
    new_url.append(new)
new_url

['https://int.api.service.nhs.uk/nhs-website-content/medicines/aciclovir/',
 'https://int.api.service.nhs.uk/nhs-website-content/medicines/acrivastine/',
 'https://int.api.service.nhs.uk/nhs-website-content/medicines/adalimumab/',
 'https://int.api.service.nhs.uk/nhs-website-content/medicines/alendronic-acid/',
 'https://int.api.service.nhs.uk/nhs-website-content/medicines/allopurinol/',
 'https://int.api.service.nhs.uk/nhs-website-content/medicines/alogliptin/',
 'https://int.api.service.nhs.uk/nhs-website-content/medicines/amitriptyline-for-depression/',
 'https://int.api.service.nhs.uk/nhs-website-content/medicines/amitriptyline-for-pain/',
 'https://int.api.service.nhs.uk/nhs-website-content/medicines/amlodipine/',
 'https://int.api.service.nhs.uk/nhs-website-content/medicines/amoxicillin/',
 'https://int.api.service.nhs.uk/nhs-website-content/medicines/anastrozole/',
 'https://int.api.service.nhs.uk/nhs-website-content/medicines/apixaban/',
 'https://int.api.service.nhs.uk/nhs-web

In [152]:
# Set the value of the substring variable
substring = 'https://int.api.service.nhs.uk/nhs-website-content/medicines'
# Create empty list
updated_url = []
# Append the first 25 entries which are formatted correctly
updated_url.append(new_url[0:25])
updated_url
# Format the remaining urls
for url in new_url:
    if substring not in url:
        temp_l, temp_r = url.split(sep = 'medicines')
        new = (substring + temp_r)
        updated_url.append(new)
updated_url

[['https://int.api.service.nhs.uk/nhs-website-content/medicines/aciclovir/',
  'https://int.api.service.nhs.uk/nhs-website-content/medicines/acrivastine/',
  'https://int.api.service.nhs.uk/nhs-website-content/medicines/adalimumab/',
  'https://int.api.service.nhs.uk/nhs-website-content/medicines/alendronic-acid/',
  'https://int.api.service.nhs.uk/nhs-website-content/medicines/allopurinol/',
  'https://int.api.service.nhs.uk/nhs-website-content/medicines/alogliptin/',
  'https://int.api.service.nhs.uk/nhs-website-content/medicines/amitriptyline-for-depression/',
  'https://int.api.service.nhs.uk/nhs-website-content/medicines/amitriptyline-for-pain/',
  'https://int.api.service.nhs.uk/nhs-website-content/medicines/amlodipine/',
  'https://int.api.service.nhs.uk/nhs-website-content/medicines/amoxicillin/',
  'https://int.api.service.nhs.uk/nhs-website-content/medicines/anastrozole/',
  'https://int.api.service.nhs.uk/nhs-website-content/medicines/apixaban/',
  'https://int.api.service.n

In [153]:
# Create empty list
med_desc = []
# Un-nest lists
for inner_list in all_desc:
    med_desc.extend(inner_list)
med_desc

['Find out how aciclovir treats cold sores, chickenpox, shingles and other herpes virus infections, and how to take it.',
 "NHS medicines information on acrivastine (Benadryl) – what it's used for, side effects, dosage and who can take it",
 'Brand names: Humira, Amgevita, Hyrimoz, Idacio, Imraldi, Yuflyma. Find out how adalimumab treats swelling (inflammation), and how to take it.',
 "NHS medicines information on alendronic acid – what it's used for, side effects, dosage and who can take it.",
 'Find out how allopurinol treats gout and how to take it.',
 'Brand name: Vipidia. Find out how alogliptin treats type 2 diabetes, and how to take it.',
 'NHS medicines information on amitriptyline for depression – how it works, side effects, dosage and who can take it.',
 'Find out how amitriptyline treats pain and prevents migraine and how to take it.',
 'Brand name: Istin. Find out how amlodipine treats high blood pressure and helps to prevent heart disease, and how to take it.',
 "NHS medic

## Transformation 1

In [156]:
# Turn list into series
api_names = pd.Series(med_names)
api_names

0      Aciclovir (Zovirax)
1              Acrivastine
2               Adalimumab
3          Alendronic acid
4              Allopurinol
              ...         
270            Solifenacin
271                Sotalol
272    Sotrovimab (Xevudy)
273         Spironolactone
274          Sulfasalazine
Length: 275, dtype: object

## Transformation 2

In [159]:
# Drop duplicates
api_names.drop_duplicates(inplace = True)
api_names

0      Aciclovir (Zovirax)
1              Acrivastine
2               Adalimumab
3          Alendronic acid
4              Allopurinol
              ...         
270            Solifenacin
271                Sotalol
272    Sotrovimab (Xevudy)
273         Spironolactone
274          Sulfasalazine
Length: 251, dtype: object

## Transformation 3

In [164]:
# Turn list into series
api_urls = pd.Series(updated_url)
api_urls

0      [https://int.api.service.nhs.uk/nhs-website-co...
1      https://int.api.service.nhs.uk/nhs-website-con...
2      https://int.api.service.nhs.uk/nhs-website-con...
3      https://int.api.service.nhs.uk/nhs-website-con...
4      https://int.api.service.nhs.uk/nhs-website-con...
                             ...                        
246    https://int.api.service.nhs.uk/nhs-website-con...
247    https://int.api.service.nhs.uk/nhs-website-con...
248    https://int.api.service.nhs.uk/nhs-website-con...
249    https://int.api.service.nhs.uk/nhs-website-con...
250    https://int.api.service.nhs.uk/nhs-website-con...
Length: 251, dtype: object

## Transformation 4

In [167]:
# Drop duplicate values
api_urls.drop_duplicates(inplace = True)
api_urls

0      [https://int.api.service.nhs.uk/nhs-website-co...
1      https://int.api.service.nhs.uk/nhs-website-con...
2      https://int.api.service.nhs.uk/nhs-website-con...
3      https://int.api.service.nhs.uk/nhs-website-con...
4      https://int.api.service.nhs.uk/nhs-website-con...
                             ...                        
246    https://int.api.service.nhs.uk/nhs-website-con...
247    https://int.api.service.nhs.uk/nhs-website-con...
248    https://int.api.service.nhs.uk/nhs-website-con...
249    https://int.api.service.nhs.uk/nhs-website-con...
250    https://int.api.service.nhs.uk/nhs-website-con...
Length: 251, dtype: object

## Transformation 5

In [170]:
# Turn into series
api_desc = pd.Series(med_desc)
api_desc

0      Find out how aciclovir treats cold sores, chic...
1      NHS medicines information on acrivastine (Bena...
2      Brand names: Humira, Amgevita, Hyrimoz, Idacio...
3      NHS medicines information on alendronic acid –...
4      Find out how allopurinol treats gout and how t...
                             ...                        
270    Find out how solifenacin treats symptoms of an...
271    NHS medicines information on sotalol – what it...
272    Find out how sotrovimab treats COVID-19 and ho...
273    Find out how spironolactone treats oedema (bui...
274    NHS medicines information on sulfasalazine – w...
Length: 275, dtype: object

## Transformation 6

In [173]:
# Drop duplicates
api_desc.drop_duplicates(inplace = True)
api_desc

0      Find out how aciclovir treats cold sores, chic...
1      NHS medicines information on acrivastine (Bena...
2      Brand names: Humira, Amgevita, Hyrimoz, Idacio...
3      NHS medicines information on alendronic acid –...
4      Find out how allopurinol treats gout and how t...
                             ...                        
270    Find out how solifenacin treats symptoms of an...
271    NHS medicines information on sotalol – what it...
272    Find out how sotrovimab treats COVID-19 and ho...
273    Find out how spironolactone treats oedema (bui...
274    NHS medicines information on sulfasalazine – w...
Length: 251, dtype: object

## Transformation 7

In [176]:
# Create empty df
api_df = pd.DataFrame()
# Assign our series to columns in the df
api_df['names'] = api_names
api_df['description'] = api_desc
api_df['url_for_more_info'] = api_urls
api_df

,names,description,url_for_more_info
0,Aciclovir (Zovirax),"Find out how aciclovir treats cold sores, chic...",[https://int.api.service.nhs.uk/nhs-website-co...
1,Acrivastine,NHS medicines information on acrivastine (Bena...,https://int.api.service.nhs.uk/nhs-website-con...
2,Adalimumab,"Brand names: Humira, Amgevita, Hyrimoz, Idacio...",https://int.api.service.nhs.uk/nhs-website-con...
3,Alendronic acid,NHS medicines information on alendronic acid –...,https://int.api.service.nhs.uk/nhs-website-con...
4,Allopurinol,Find out how allopurinol treats gout and how t...,https://int.api.service.nhs.uk/nhs-website-con...
...,...,...,...
270,Solifenacin,Find out how solifenacin treats symptoms of an...,NaN
271,Sotalol,NHS medicines information on sotalol – what it...,NaN
272,Sotrovimab (Xevudy),Find out how sotrovimab treats COVID-19 and ho...,NaN
273,Spironolactone,Find out how spironolactone treats oedema (bui...,NaN


## Transformation 8

In [179]:
# Drop any rows with na
api_df.dropna(inplace = True)
api_df

,names,description,url_for_more_info
0,Aciclovir (Zovirax),"Find out how aciclovir treats cold sores, chic...",[https://int.api.service.nhs.uk/nhs-website-co...
1,Acrivastine,NHS medicines information on acrivastine (Bena...,https://int.api.service.nhs.uk/nhs-website-con...
2,Adalimumab,"Brand names: Humira, Amgevita, Hyrimoz, Idacio...",https://int.api.service.nhs.uk/nhs-website-con...
3,Alendronic acid,NHS medicines information on alendronic acid –...,https://int.api.service.nhs.uk/nhs-website-con...
4,Allopurinol,Find out how allopurinol treats gout and how t...,https://int.api.service.nhs.uk/nhs-website-con...
...,...,...,...
246,Promethazine (Phenergan),NHS medicines information on promethazine – wh...,https://int.api.service.nhs.uk/nhs-website-con...
247,Propranolol,NHS medicines information on propranolol – wha...,https://int.api.service.nhs.uk/nhs-website-con...
248,Pseudoephedrine (Sudafed),Find out how pseudoephedrine treats a blocked ...,https://int.api.service.nhs.uk/nhs-website-con...
249,Quetiapine,Find out how quetiapine treats mental health c...,https://int.api.service.nhs.uk/nhs-website-con...


The data came in dictionary/json format. I created lists that I then turned into pandas series to perform operations on. I dropped all of the duplicates and then combined all the data into a single dataframe. I dropped the rows which had NaN since we're using this as supplementary data and missing values would potentially impede a model. There is the HIPAA law but it doesn't apply to us at this stage of the data. The data was sourced from the NHS which is the universal healthcare system across the pond, so I feel comfortable placing my name on the accuracy of their data.